Imports

In [1]:
import pandas as pd
from congress import Congress 
congress = Congress('your api key')
import zipfile
from zipfile import ZipFile
import json 
import threading 
import timeit
import time 
import numpy as np 
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from multiprocessing import Pool 

In [6]:
pd.set_option('display.max_columns', None)

Functions

In [3]:
def committee_compiler(congress_list):
    file_list = []
    for congress in congress_list:
        file_list.append('data_store/committees/congress_' + str(
            congress) + '_committees.csv')
    each_df = (pd.read_csv(file) for file in file_list)
    combined_df = pd.concat(each_df, ignore_index=True)
    return combined_df 

In [2]:
def member_compiler(congress_list, chamber):
    file_list = []
    for congress in congress_list:
        file_list.append('data_store/members/' + str(
            chamber) +'_'  + str(congress) + '_members.csv')
    each_df = (pd.read_csv(file) for file in file_list)
    combined_df = pd.concat(each_df, ignore_index=True)
    return combined_df 

In [5]:
def file_name_compiler(congress_list):
    congress = []
    file_list = []
    for congress_number in congress_list:
        with zipfile.ZipFile('bill_metadata/' + str(congress_number) + '.zip', 'r') as z:
                for file in z.namelist():
                    if congress_number > 112:
                        if file.startswith('congress/data/' +str(
                            congress_number) + '/bills/hr/hr') & file.endswith(
                            'data.json'):
                            file_list.append(file)
                            congress.append(congress_number)
                        elif file.startswith('congress/data/' +str(
                            congress_number) + '/bills/s/s') & file.endswith(
                            'data.json'):
                            file_list.append(file)
                            congress.append(congress_number)
                        else:
                            pass
                    else:
                        if file.startswith('bills/hr/hr') & file.endswith(
                            'data.json'):
                            file_list.append(file)
                            congress.append(congress_number)
                        elif file.startswith('bills/s/s') & file.endswith(
                            'data.json'):
                            file_list.append(file)
                            congress.append(congress_number)
                        else:
                            pass
    data_dict = {'congress': congress, 'file_name': file_list}
    df = pd.DataFrame(data=data_dict)
    return df 

In [6]:
def metadata_compiler(file_df, bill_metadata_name, output_list, process_function): #, process_function):
    for congress_number in file_df['congress'].unique():
        with zipfile.ZipFile('bill_metadata/' + str(congress_number) + '.zip', 'r') as z:
            for file in file_df['file_name'][
                file_df['congress'] == congress_number].reset_index(drop=True):
                with z.open(file) as f:
                    data = f.read()
                    d = json.loads(data.decode('utf-8'))
                    if bill_metadata_name in d.keys():
                        error_var = 0 
                        input_list = d[bill_metadata_name]
                        process_function(output_list, input_list, d, error_var)
                    else:
                        error_var = 1
                        input_list = 'feature_error'
                        process_function(output_list, input_list, d, error_var)
                        output_list.append('feature_error')

In [7]:
def status_compiler(file_df, bill_metadata_name, output_list, process_function, status): #, process_function):
    for congress_number in file_df['congress'].unique():
        with zipfile.ZipFile('bill_metadata/' + str(congress_number) + '.zip', 'r') as z:
            for file in file_df['file_name'][
                file_df['congress'] == congress_number].reset_index(drop=True):
                with z.open(file) as f:
                    data = f.read()
                    d = json.loads(data.decode('utf-8'))
                    if bill_metadata_name in d.keys():
                        error_var = 0 
                        input_list = d[bill_metadata_name]
                        process_function(output_list, input_list, d, error_var, status)
                    else:
                        error_var = 1
                        input_list = 'feature_error'
                        process_function(output_list, input_list, d, error_var, status)
                        output_list.append('feature_error')

In [30]:
pre_vote_action_test = metadata_compiler(file_df=df,
                                         bill_metadata_name='actions',
                                         output_list=empty_list,
                                         process_function=pre_vote_actions)

In [43]:
unique = []
for act in range(len(df_pre_vote_2_actions)):
    if df_pre_vote_2_actions[act] in unique:
        pass
    else:
        unique.append(df_pre_vote_2_actions[act])
        

In [28]:
df_pre_vote_1_actions = []
df_pre_vote_2_actions = []

def pre_vote_actions(output_list, input_list, d, error_var):
    if error_var == 0:
        pre_vote_1_actions = 0
        pre_vote_2_actions = 0
        for action in range(len(input_list)):
            if 'vote_type' in input_list[action]:
                if input_list[action]['vote_type'] == 'vote':
                    pre_vote_1_actions = action - 1
                else:
                    pass
                if input_list[action]['vote_type'] == 'vote2':
                    pre_vote_2_actions = action - 1
                else:
                    pass
            else:
                pass
        df_pre_vote_1_actions.append(pre_vote_1_actions)
        df_pre_vote_2_actions.append(pre_vote_2_actions)
    else:
        output_list.append('feature_error')            

In [8]:
def count_aggregator(output_list, input_list, d, error_var):
    if error_var == 0: 
        output_list.append(len(input_list))
    else:
        output_list.append('feature_error')

In [10]:
def binary_check(output_list, input_list, d, error_var):
    if error_var == 0:
        if input_list == True:
            output_list.append(1)
        else:
            output_list.append(0)
    else:
        output_list.append('feature_error')

In [11]:
def return_value(output_list, input_list, d, error_var):
    if error_var == 0:
        output_list.append(input_list)
    else: 
        output_list.append('feature_error')

In [12]:
def return_summary_txt(output_list, input_list, d, error_var):
    if error_var == 0:
        if input_list is None:
            output_list.append('No summary') 
        else:
            output_list.append(input_list['text'])
    else: 
        output_list.append('feature_error')

In [13]:
def committee_activity_check(output_list, input_list, d, error_var):
    
    if error_var == 0:
        congress_number = int(d['congress'])
    
        current_committees = committees[
            committees['congress_number'] == congress_number]
    
        referral_committees = [referral_committee_1, referral_committee_2,
                               referral_committee_3]
    
        markup_committees = [markup_committee_1, markup_committee_2,
                               markup_committee_3]
    
        reporting_committees = [reporting_committee_1, reporting_committee_2,
                               reporting_committee_3]
    
        hearings_committees = [hearings_committee_1, hearings_committee_2,
                               hearings_committee_3]
    
        unused_committees = []
        used_committees = []
    
        for committee in range(3):
            if committee not in range(len(input_list)):
                unused_committees.append(committee)
            else:
                used_committees.append(committee)
            
        for used_committee in used_committees:    
            if ('referral' in input_list[used_committee]['activity']):
                referral_committees[used_committee].append(
                    input_list[used_committee]['committee_id'])
            else:
                referral_committees[used_committee].append(np.nan)
            
            if ('markup' in input_list[used_committee]['activity']):
                markup_committees[used_committee].append(
                    input_list[used_committee]['committee_id'])
            else:
                markup_committees[used_committee].append(np.nan)
            
            if ('reporting' in input_list[used_committee]['activity']): 
                reporting_committees[used_committee].append(
                    input_list[used_committee]['committee_id'])
            else:
                reporting_committees[used_committee].append(np.nan)
            
            if ('hearings' in input_list[used_committee]['activity']): 
                hearings_committees[used_committee].append(
                    input_list[used_committee]['committee_id'])
            else:
                hearings_committees[used_committee].append(np.nan)
            
        if len(unused_committees) > 0:
            for unused_committee in unused_committees:
                referral_committees[unused_committee].append(np.nan) 
                markup_committees[unused_committee].append(np.nan)
                reporting_committees[unused_committee].append(np.nan)
                hearings_committees[unused_committee].append(np.nan)
        else: 
            pass
    else: 
        referral_committee_1.append('feature_error')
        referral_committee_2.append('feature_error')
        referral_committee_3.append('feature_error')
        markup_committee_1.append('feature_error')
        markup_committee_2.append('feature_error')
        markup_committee_3.append('feature_error')
        reporting_committee_1.append('feature_error')
        reporting_committee_2.append('feature_error') 
        reporting_committee_3.append('feature_error')
        hearings_committee_1.append('feature_error')
        hearings_committee_2.append('feature_error')
        hearings_committee_3.append('feature_error')
    
    #if user_input in current_committees['id']:
    #    df_own_committee_check.append(1)
    #else:
    #    df_own_committee_check.append(0)

In [14]:
def sponsor_functions(output_list, input_list, d, error_var):
    
    if error_var == 0:
        congress_number = int(d['congress'])
        
        if len(input_list) > 0:
            if congress_number > 112:
                    if input_list['bioguide_id'] in list(r_df['bioguide_id']):
                        output_list.append(1) 
                    else:
                        output_list.append(0) 
            else:
                    if input_list['thomas_id'] in list(r_df['thomas_id']):
                        output_list.append(1) 
                    else:
                         output_list.append(0)
        else:
            output_list.append('None')
    else: 
        output_list.append('feature_error')

In [15]:
def cosponsor_functions(output_list, input_list, d, error_var):
    
    
    if error_var == 0:
        congress_number = int(d['congress'])
        
        r_count = 0 
        d_count = 0
    
        is_cosponsor = 0
        
        if len(input_list) > 0:
            if congress_number > 112:
                for i in range(len(input_list)):
            #cosponsors = input_list[i]['bioguide_id']
                    if input_list[i]['bioguide_id'] in list(r_df['bioguide_id']):
                        r_count +=1 
                    else:
                        pass 
            else:
                for i in range(len(input_list)):
            #cosponsors = input_list[i]['bioguide_id'] 
                    if input_list[i]['thomas_id'] in list(r_df['thomas_id']):
                        r_count +=1 
                    else:
                        pass
        else:
            output_list.append('None')

        d_count = len(input_list) - r_count  
    
        df_r_cosponsors.append(r_count)
        df_d_cosponsors.append(d_count)
        df_cosponsor_count.append(len(input_list))
    
    else: 
        df_r_cosponsors.append('feature_error')
        df_d_cosponsors.append('feature_error')
        df_cosponsor_count.append('feature_error') 

In [16]:
def chamber_partisan_check(output_list, input_list, d, error_var):
    if error_var == 0:
        congress_number = int(d['congress'])
        
        composition_df = chamber_composition_df[
            chamber_composition_df['congress'] == congress_number]
        
        house_republicans.append(composition_df['house_Rs'].values[0])
        house_democrats.append(composition_df['house_Ds'].values[0])
        senate_republicans.append(composition_df['senate_Rs'].values[0])
        senate_democrats.append(composition_df['senate_Ds'].values[0])
        
    else:
        output_list.append('feature_error')

In [17]:
def action_check(output_list, input_list, d, error_var, status):
    if error_var == 0:
        status_val = 0
        for i in range(len(input_list)): 
            if 'status' in input_list[i]:
                if (input_list[i]['status']) == str(status):
                    status_val +=1 
                    pass 
            else:
                pass 
        output_list.append(status_val)
    else:
        output_list.append('feature_error')

In [18]:
def roll_check(output_list, input_list, d, error_var, status):
    if error_var == 0:
        status_val = None
        for i in range(len(input_list)): 
            if 'status' in input_list[i]:
                if (input_list[i]['status']) == str(status):
                    if 'roll' in input_list[i]:
                        status_val = input_list[i]['roll']
                        pass
                    else:
                        pass
                else:
                    pass
            else:
                pass
        output_list.append(status_val)
    else:
        output_list.append('feature_error')

In [19]:
def subject_check(output_list, input_list, d, error_var):
    if error_var == 0: 
        
        for i in range(len(input_list)):
            if '' in input_list[i]:
                if 'PASS_OVER:HOUSE' in input_list[i]['status']:
                    output_list.append('GOT EEM')
                else:
                    pass
            else:
                pass
    else:
        output_list.append('feature_error')

Reading in Data 

In [20]:
all_congress_numbers = list(range(93, 117))  
committees = committee_compiler(all_congress_numbers) 
senate = member_compiler(all_congress_numbers, 'senate') 
house = member_compiler(all_congress_numbers, 'house')
congress_members = house.append(senate, sort=False) 
legislators = pd.read_csv('all_legislators.csv')
r_df = legislators[legislators['party'] == 'Republican']
df = file_name_compiler(all_congress_numbers) 
test_93 = file_name_compiler([93, 94])

In [4]:
all_congress_numbers = list(range(93, 117)) 
senate = member_compiler(all_congress_numbers, 'senate') 
house = member_compiler(all_congress_numbers, 'house')
congress_members = house.append(senate, sort=False) 

In [5]:
congress_members.columns

Index(['api_uri', 'at_large', 'congress_number', 'contact_form', 'crp_id',
       'cspan_id', 'date_of_birth', 'district', 'dw_nominate',
       'facebook_account', 'fax', 'fec_candidate_id', 'first_name', 'gender',
       'geoid', 'google_entity_id', 'govtrack_id', 'icpsr_id', 'id',
       'ideal_point', 'in_office', 'last_name', 'last_updated',
       'leadership_role', 'middle_name', 'missed_votes', 'missed_votes_pct',
       'next_election', 'ocd_id', 'office', 'party', 'phone', 'rss_url',
       'seniority', 'short_title', 'state', 'suffix', 'title', 'total_present',
       'total_votes', 'twitter_account', 'url', 'votes_with_party_pct',
       'votesmart_id', 'youtube_account', 'chamber', 'lis_id', 'senate_class',
       'state_rank'],
      dtype='object')

In [7]:
congress_members.head()

,api_uri,at_large,congress_number,contact_form,crp_id,cspan_id,date_of_birth,district,dw_nominate,facebook_account,fax,fec_candidate_id,first_name,gender,geoid,google_entity_id,govtrack_id,icpsr_id,id,ideal_point,in_office,last_name,last_updated,leadership_role,middle_name,missed_votes,missed_votes_pct,next_election,ocd_id,office,party,phone,rss_url,seniority,short_title,state,suffix,title,total_present,total_votes,twitter_account,url,votes_with_party_pct,votesmart_id,youtube_account,chamber,lis_id,senate_class,state_rank
0,https://api.propublica.org/congress/v1/members...,False,102,NaN,NaN,NaN,1938-06-26,1,NaN,NaN,NaN,NaN,Neil,M,1501,/m/0255rj,400001.0,15244.0,A000014,NaN,False,Abercrombie,2016-06-14 16:25:53 -0400,NaN,NaN,30.0,3.22,NaN,ocd-division/country:us/state:hi/cd:1,NaN,D,NaN,NaN,4,Rep.,HI,NaN,Representative,0.0,932.0,neilabercrombie,NaN,87.03,NaN,hawaiirep1,h,NaN,NaN,NaN
1,https://api.propublica.org/congress/v1/members...,False,102,NaN,NaN,1002061.0,1942-11-19,7,NaN,NaN,NaN,NaN,Gary,M,3607,/m/03rhdq,400003.0,15080.0,A000022,NaN,False,Ackerman,2016-06-14 16:25:53 -0400,NaN,L.,173.0,18.56,NaN,ocd-division/country:us/state:ny/cd:7,NaN,D,NaN,NaN,10,Rep.,NY,NaN,Representative,0.0,932.0,repgaryackerman,NaN,89.86,NaN,RepAckerman,h,NaN,NaN,NaN
2,https://api.propublica.org/congress/v1/members...,False,102,NaN,NaN,NaN,1934-01-16,1,NaN,NaN,NaN,NaN,William,M,501,NaN,400761.0,12000.0,A000103,NaN,False,Alexander,2013-04-12 20:11:55 -0400,NaN,Vollie,188.0,20.17,NaN,ocd-division/country:us/state:ar/cd:1,NaN,D,NaN,NaN,24,Rep.,AR,NaN,Representative,0.0,932.0,NaN,NaN,86.29,NaN,NaN,h,NaN,NaN,NaN
3,https://api.propublica.org/congress/v1/members...,False,102,NaN,NaN,NaN,1943-12-02,4,NaN,NaN,NaN,NaN,Wayne,M,804,NaN,300003.0,15801.0,A000109,NaN,False,Allard,2013-04-12 20:12:01 -0400,NaN,A.,16.0,1.72,NaN,ocd-division/country:us/state:co/cd:4,NaN,R,NaN,NaN,2,Rep.,CO,NaN,Representative,0.0,932.0,NaN,NaN,82.21,NaN,NaN,h,NaN,NaN,NaN
4,https://api.propublica.org/congress/v1/members...,False,102,NaN,NaN,NaN,1952-03-08,7,NaN,NaN,NaN,NaN,George,M,5107,NaN,300004.0,15847.0,A000121,NaN,False,Allen,2013-04-12 20:12:00 -0400,NaN,Felix,4.0,0.73,NaN,ocd-division/country:us/state:va/cd:7,NaN,R,NaN,NaN,2,Rep.,VA,NaN,Representative,1.0,545.0,NaN,NaN,83.55,NaN,NaN,h,NaN,NaN,NaN


In [8]:
legislators = pd.read_csv('all_legislators.csv') 

In [10]:
test = pd.read_csv('data_store/members/all_legislators.csv') 

In [11]:
test['bioguide_id'].nunique() 

12516

In [9]:
legislators['bioguide_id'].nunique() 

12514

In [21]:
congress_numbers = [93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107,
                   108, 109, 110, 111, 112, 113, 114, 115, 116] 

senate_Rs = [42, 37, 38, 41, 53, 54, 53, 45, 34, 44, 43, 53, 55, 55, 50, 51, 55, 49, 42,
            47, 45, 54, 51, 53]

senate_Ds = [56, 61, 61, 58, 46, 46, 47, 55, 55, 56, 57, 47, 45, 45, 50, 48, 44, 49, 58,
            51, 53, 44, 47, 45]

house_Rs = [192, 144, 143, 158, 192, 166, 182, 177, 175, 167, 176, 230, 227, 223, 221, 229,
            232, 202, 178, 242, 234, 247, 241, 198]

house_Ds = [242, 291, 292, 277, 242, 269, 253, 258, 260, 267, 258, 204, 206, 211, 212, 205,
           202, 233, 257, 193, 201, 188, 194, 235]

In [22]:
chamber_parties_dict = {'congress': congress_numbers, 'senate_Rs': senate_Rs,
                       'senate_Ds': senate_Ds, 'house_Rs': house_Rs,
                       'house_Ds': house_Ds}

In [23]:
chamber_composition_df = pd.DataFrame(chamber_parties_dict)

In [24]:
chamber_composition_df

,congress,senate_Rs,senate_Ds,house_Rs,house_Ds
0,93,42,56,192,242
1,94,37,61,144,291
2,95,38,61,143,292
3,96,41,58,158,277
4,97,53,46,192,242
5,98,54,46,166,269
6,99,53,47,182,253
7,100,45,55,177,258
8,101,34,55,175,260
9,102,44,56,167,267


In [25]:
congress_members.columns

Index(['api_uri', 'at_large', 'congress_number', 'contact_form', 'crp_id',
       'cspan_id', 'date_of_birth', 'district', 'dw_nominate',
       'facebook_account', 'fax', 'fec_candidate_id', 'first_name', 'gender',
       'geoid', 'google_entity_id', 'govtrack_id', 'icpsr_id', 'id',
       'ideal_point', 'in_office', 'last_name', 'last_updated',
       'leadership_role', 'middle_name', 'missed_votes', 'missed_votes_pct',
       'next_election', 'ocd_id', 'office', 'party', 'phone', 'rss_url',
       'seniority', 'short_title', 'state', 'suffix', 'title', 'total_present',
       'total_votes', 'twitter_account', 'url', 'votes_with_party_pct',
       'votesmart_id', 'youtube_account', 'chamber', 'lis_id', 'senate_class',
       'state_rank'],
      dtype='object')

In [26]:
legislators['bioguide_id'].isna().sum()

0

In [26]:
legislators 

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,district,senate_class,party,url,address,phone,contact_form,rss_url,twitter,facebook,youtube,youtube_id,bioguide_id,thomas_id,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,NaN,1.0,Democrat,https://www.brown.senate.gov,503 Hart Senate Office Building Washington DC ...,202-224-2315,http://www.brown.senate.gov/contact/,http://www.brown.senate.gov/rss/feeds/?type=al...,SenSherrodBrown,SenatorSherrodBrown,SherrodBrownOhio,UCgy8jfERh-t_ixkKKoCmglQ,B000944,136.0,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,NaN,1.0,Democrat,https://www.cantwell.senate.gov,511 Hart Senate Office Building Washington DC ...,202-224-3441,http://www.cantwell.senate.gov/public/index.cf...,http://www.cantwell.senate.gov/public/index.cf...,SenatorCantwell,senatorcantwell,SenatorCantwell,UCN52UDqKgvHRk39ncySrIMw,C000127,172.0,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,NaN,1.0,Democrat,https://www.cardin.senate.gov,509 Hart Senate Office Building Washington DC ...,202-224-4524,http://www.cardin.senate.gov/contact/,http://www.cardin.senate.gov/rss/feeds/?type=all,SenatorCardin,senatorbencardin,senatorcardin,UCiQaJnMzlfzzG3VESgyZChA,C000141,174.0,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin
3,Carper,Thomas,Richard,NaN,NaN,Thomas R. Carper,1947-01-23,M,sen,DE,NaN,1.0,Democrat,https://www.carper.senate.gov/public,513 Hart Senate Office Building Washington DC ...,202-224-2441,http://www.carper.senate.gov/public/index.cfm/...,http://www.carper.senate.gov/public/index.cfm/...,SenatorCarper,tomcarper,senatorcarper,UCgLnvbKwu4B3navofj6Qvvw,C000174,179.0,N00012508,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,NaN,1.0,Democrat,https://www.casey.senate.gov,393 Russell Senate Office Building Washington ...,202-224-6324,https://www.casey.senate.gov/contact/,http://www.casey.senate.gov/rss/feeds/?all,SenBobCasey,SenatorBobCasey,SenatorBobCasey,UCtVssXhx-KuZa-hSvnsnJ0A,C001070,1828.0,N00027503,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.
5,Feinstein,Dianne,NaN,NaN,NaN,Dianne Feinstein,1933-06-22,F,sen,CA,NaN,1.0,Democrat,https://www.feinstein.senate.gov,331 Hart Senate Office Building Washington DC ...,202-224-3841,https://www.feinstein.senate.gov/public/index....,http://www.feinstein.senate.gov/public/?a=rss....,SenFeinstein,senatorfeinstein,SenatorFeinstein,UCtVC--6LR0ff2aOP8THpuEw,F000062,1332.0,N00007364,S221,S0CA00199,13061.0,300043,53273.0,Dianne Feinstein,NaN,49300.0,Dianne Feinstein
6,Klobuchar,Amy,Jean,NaN,NaN,Amy Klobuchar,1960-05-25,F,sen,MN,NaN,1.0,Democrat,https://www.klobuchar.senate.gov,425 Dirksen Senate Office Building Washington ...,202-224-3244,http://www.klobuchar.senate.gov/public/index.c...,NaN,SenAmyKlobuchar,NaN,senatorklobuchar,UCvdeJsDsV51tFb_hVqvtYGA,K000367,1826.0,N00027500,S311,S6MN00267,83701.0,412242,65092.0,Amy Klobuchar,NaN,40700.0,Amy Klobuchar
7,Menendez,Robert,NaN,NaN,Bob,Robert Menendez,1954-01-01,M,sen,NJ,NaN,1.0,Democrat,https://www.menendez.senate.gov,528 Hart Senate Office Building Washington DC ...,202-224-4744,https://www.menendez.senate.gov/contact,http://www.menendez.senate.gov/rss/feeds/index...,SenatorMenendez,senatormenendez,SenatorMenendezNJ,UC0PV0K9Z5a9p3D5917KF5fw,M000639,791.0,N00000699,S306,"H2NJ13075,S6NJ00289",29608.0,400272,26961.0,Bob Menendez,NaN,29373.0,Bob Menendez
8,Sanders,Bernard,NaN,NaN,Bernie,Bernard Sanders,1941-09-08,M,sen,VT,NaN,1.0,Independent,https://www.sanders.senate.gov,332 

In [27]:
empty_committee_list = []
empty_cosponsor_list = []
empty_sponsor_list = [] 

df_action_count = [] #Error
df_r_sponsor = []
df_amendment_count = [] #Error
df_by_request = []#Error
df_committee_count = [] #Error
df_cosponsor_count = []
df_r_cosponsors = [] 
df_d_cosponsors = []
df_popular_title = [] #Error
df_related_bill_count = [] #Error
df_short_title = [] #Error
df_subject_count = [] #Error
df_title_count = [] #Error
df_status = []
df_subject_1 = []
df_subject_2 = []
df_subject_3 = []
df_bill_number = []  #"number"

house_republicans = []
house_democrats = []
senate_republicans = []
senate_democrats = []

referral_committee_1 = []
referral_committee_2 = []
referral_committee_3 = []
markup_committee_1 = []
markup_committee_2 = []
markup_committee_3 = []
reporting_committee_1 = []
reporting_committee_2 = []
reporting_committee_3 = []
hearings_committee_1 = [] 
hearings_committee_2 = []
hearings_committee_3 = []

Function Tests 

In [26]:
all_congress_numbers = list(range(93, 117))  

In [27]:

#test_congress_numbers = [104] 

df = file_name_compiler(all_congress_numbers) 

In [290]:
len(df)

255449

In [291]:
df.head(1)

,congress,file_name
0,93,bills/s/s2009/data.json


Committee Activity Check Tests

In [307]:
%%timeit 
metadata_compiler(df, 'committees', empty_committee_list, committee_activity_check) 

2min 25s ± 1.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Cosponsor Function Checks 

In [356]:
metadata_compiler(df, 'cosponsors', empty_cosponsor_list, cosponsor_functions) 

In [279]:
metadata_compiler(df, 'actions', action_count_1, count_aggregator)

Chamber Composition Checks

In [177]:
metadata_compiler(df, 'congress', empty_committee_list, chamber_partisan_check) 

Return Value Test 

In [28]:

metadata_compiler(df, 'bill_id', bill_numbers, return_value)

In [46]:
len(bill_numbers)

255449

In [36]:
bill_id = []
bill_type = []
congress = []
number = []

metadata_compiler(df, 'bill_id', bill_id, return_value)
metadata_compiler(df, 'bill_type', bill_type, return_value) 
metadata_compiler(df, 'congress', congress, return_value) 
metadata_compiler(df, 'number', number, return_value) 

print ('bill ids:', len(bill_id))
print ('bill types:', len(bill_type))
print ('bill congress:', len(congress))
print ('bill numbers:', len(number))

bill ids: 255449
bill types: 255449
bill congress: 255449
bill numbers: 255449


Return Summary Text

In [43]:
summary_text = []
metadata_compiler(df, 'summary', summary_text, return_summary_txt)

In [45]:
len(summary_text)

255449

Action Check Tests

In [55]:
action_test_list = []

In [56]:
# The action checker actually works surprisingly well, but the data discrepancies are every
# bit as bad as the GPO said- maybe even worse, so its value might be negligible 
metadata_compiler(test_93, 'actions', action_test_list, action_check)

Subject List Checks 

In [182]:
list_of_outputs = [df_action_count, 
                   df_r_sponsor, 
                   df_amendment_count,  
                   df_committee_count, 
                   df_cosponsor_count, 
                   df_r_cosponsors, 
                   df_d_cosponsors, 
                   df_popular_title, 
                   df_related_bill_count, 
                   df_short_title, 
                   df_subject_count, 
                   df_title_count, 
                   house_republicans,
                   house_democrats,
                   senate_republicans,
                   senate_democrats ,
                   referral_committee_1, 
                   referral_committee_2,
                   referral_committee_3,
                   markup_committee_1,
                   markup_committee_2,
                   markup_committee_3,
                   reporting_committee_1,
                   reporting_committee_2,
                   reporting_committee_3,
                   hearings_committee_1,
                   hearings_committee_2,
                   hearings_committee_3,
                   df_status]

In [183]:
list_of_col_names = ['action_count', 
                   'r_sponsor', 
                   'amendment_count',  
                   'committee_count', 
                   'cosponsor_count', 
                   'r_cosponsors', 
                   'd_cosponsors', 
                   'popular_title', 
                   'related_bill_count', 
                   'short_title', 
                   'subject_count', 
                   'title_count', 
                   'house_republicans',
                   'house_democrats',
                   'senate_republicans',
                   'senate_democrats',
                   'referral_committee_1', 
                   'referral_committee_2',
                   'referral_committee_3',
                   'markup_committee_1',
                   'markup_committee_2',
                   'markup_committee_3',
                   'reporting_committee_1',
                   'reporting_committee_2',
                   'reporting_committee_3',
                   'hearings_committee_1',
                   'hearings_committee_2',
                   'hearings_committee_3',
                    'status']

In [184]:
for i in list_of_outputs:
    print (len(i)) 

255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449
255449


# BILL DF COLUMN ASSEMBLY

In [108]:
metadata_compiler(df, 'actions', df_action_count, count_aggregator) 

In [109]:
metadata_compiler(df, 'sponsor', df_r_sponsor, sponsor_functions) 

In [110]:
metadata_compiler(df, 'amendments', df_amendment_count, count_aggregator) 

In [133]:
metadata_compiler(df, 'by_request', df_by_request, binary_check) 

In [112]:
metadata_compiler(df, 'committees', df_committee_count, count_aggregator) 

In [113]:
metadata_compiler(df, 'cosponsors', empty_cosponsor_list, cosponsor_functions) 

In [114]:
metadata_compiler(df, 'popular_title', df_popular_title, binary_check) 

In [115]:
metadata_compiler(df, 'related_bills', df_related_bill_count, count_aggregator)  

In [116]:
metadata_compiler(df, 'short_title', df_short_title, binary_check) 

In [117]:
metadata_compiler(df, 'subjects', df_subject_count, count_aggregator)  

In [118]:
metadata_compiler(df, 'titles', df_title_count, count_aggregator) 

In [119]:
metadata_compiler(df, 'status', df_status, return_value)  

#### For subjects, I just need to merge the subject df with the bill df once it is done 

In [120]:
metadata_compiler(df, 'number', df_bill_number, return_value)  

In [121]:
metadata_compiler(df, 'congress', empty_committee_list, chamber_partisan_check) 

In [122]:
metadata_compiler(df, 'committees', empty_committee_list, committee_activity_check)

# BILL DF ASSEMBLY 

In [185]:
congress_level_df = pd.DataFrame(list_of_outputs).T 

In [186]:
congress_level_df.columns = list_of_col_names

In [188]:
congress_level_df.head()

,action_count,r_sponsor,amendment_count,committee_count,cosponsor_count,r_cosponsors,d_cosponsors,popular_title,related_bill_count,short_title,subject_count,title_count,house_republicans,house_democrats,senate_republicans,senate_democrats,referral_committee_1,referral_committee_2,referral_committee_3,markup_committee_1,markup_committee_2,markup_committee_3,reporting_committee_1,reporting_committee_2,reporting_committee_3,hearings_committee_1,hearings_committee_2,hearings_committee_3,status
0,1,0,0,1,0,0,0,0,0,0,2,1,192,242,42,56,SSFI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
1,1,0,0,0,7,0,7,0,0,0,3,2,192,242,42,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTRODUCED
2,1,0,0,1,0,0,0,0,1,0,4,1,192,242,42,56,SSEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
3,1,0,0,1,0,0,0,0,0,0,2,1,192,242,42,56,SSFI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
4,3,0,0,1,4,0,4,0,1,0,4,2,192,242,42,56,SSHR,NaN,NaN,NaN,NaN,NaN,SSHR,NaN,NaN,NaN,NaN,NaN,REFERRED


In [190]:
congress_level_df.to_csv('congress_level_df.csv', index=False)

In [32]:
congress_level_df = pd.read_csv('data_store/dataframes/congress_level_df.csv')

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,20,21,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
congress_level_df.head()

,action_count,r_sponsor,amendment_count,committee_count,cosponsor_count,r_cosponsors,d_cosponsors,popular_title,related_bill_count,short_title,subject_count,title_count,house_republicans,house_democrats,senate_republicans,senate_democrats,referral_committee_1,referral_committee_2,referral_committee_3,markup_committee_1,markup_committee_2,markup_committee_3,reporting_committee_1,reporting_committee_2,reporting_committee_3,hearings_committee_1,hearings_committee_2,hearings_committee_3,status
0,1,0,0,1,0,0,0,0,0,0,2,1,192,242,42,56,SSFI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
1,1,0,0,0,7,0,7,0,0,0,3,2,192,242,42,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTRODUCED
2,1,0,0,1,0,0,0,0,1,0,4,1,192,242,42,56,SSEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
3,1,0,0,1,0,0,0,0,0,0,2,1,192,242,42,56,SSFI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
4,3,0,0,1,4,0,4,0,1,0,4,2,192,242,42,56,SSHR,NaN,NaN,NaN,NaN,NaN,SSHR,NaN,NaN,NaN,NaN,NaN,REFERRED


In [37]:
id_cols = [bill_id, bill_type, congress, number]
id_col_names = ['bill_id', 'bill_type', 'congress', 'number']

bill_id_df = pd.DataFrame(id_cols).T

In [38]:
bill_id_df.columns = id_col_names 

In [40]:
bill_id_df.head()

,bill_id,bill_type,congress,number
0,s2009-93,s,93,2009
1,s4114-93,s,93,4114
2,s225-93,s,93,225
3,s3969-93,s,93,3969
4,s3344-93,s,93,3344


In [42]:
congress_level_df = pd.concat([bill_id_df, congress_level_df], axis=1) 

In [43]:
congress_level_df.head(5)

,bill_id,bill_type,congress,number,action_count,r_sponsor,amendment_count,committee_count,cosponsor_count,r_cosponsors,d_cosponsors,popular_title,related_bill_count,short_title,subject_count,title_count,house_republicans,house_democrats,senate_republicans,senate_democrats,referral_committee_1,referral_committee_2,referral_committee_3,markup_committee_1,markup_committee_2,markup_committee_3,reporting_committee_1,reporting_committee_2,reporting_committee_3,hearings_committee_1,hearings_committee_2,hearings_committee_3,status
0,s2009-93,s,93,2009,1,0,0,1,0,0,0,0,0,0,2,1,192,242,42,56,SSFI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
1,s4114-93,s,93,4114,1,0,0,0,7,0,7,0,0,0,3,2,192,242,42,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INTRODUCED
2,s225-93,s,93,225,1,0,0,1,0,0,0,0,1,0,4,1,192,242,42,56,SSEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
3,s3969-93,s,93,3969,1,0,0,1,0,0,0,0,0,0,2,1,192,242,42,56,SSFI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REFERRED
4,s3344-93,s,93,3344,3,0,0,1,4,0,4,0,1,0,4,2,192,242,42,56,SSHR,NaN,NaN,NaN,NaN,NaN,SSHR,NaN,NaN,NaN,NaN,NaN,REFERRED


In [44]:
subject_df = pd.read_csv('coded_subject_data.csv') 

In [45]:
subject_df.head(5)

,bill_number,summary,major_label 1,major_prob 1,major_label 2,major_prob 2,major_label 3,major_prob 3,major_1_minor_label 1,major_1_minor_prob 1,major_1_minor_label 2,major_1_minor_prob 2,major_1_minor_label 3,major_1_minor_prob 3,major_2_minor_label 1,major_2_minor_prob 1,major_2_minor_label 2,major_2_minor_prob 2,major_2_minor_label 3,major_2_minor_prob 3,major_3_minor_label 1,major_3_minor_prob 1,major_3_minor_label 2,major_3_minor_prob 2,major_3_minor_label 3,major_3_minor_prob 3
0,s2009-93,Provides that whenever the Secretary of the Tr...,18.0,64.576,20.0,4.833,7.0,4.428,1807.0,86.326,1800.0,3.585,1802.0,2.849,2008.0,16.105,2006.0,13.966,2000.0,11.424,710.0,38.059,709.0,19.673,705.0,13.267
1,s4114-93,Emergency Budget Control Act - Authorizes the ...,20.0,62.421,1.0,13.264,15.0,4.668,2011.0,46.635,2002.0,22.675,2000.0,21.265,105.0,93.410,101.0,1.603,108.0,1.545,1523.0,56.273,1520.0,8.700,1521.0,8.091
2,s225-93,"Authorizes the project for the Days Creek Dam,...",21.0,68.645,20.0,5.221,15.0,3.689,2104.0,93.032,2100.0,2.191,2103.0,1.768,2008.0,53.425,2001.0,10.555,2002.0,10.522,1523.0,59.422,1599.0,15.701,1502.0,5.735
3,s3969-93,Requires a one-year study of the feasibility o...,13.0,75.467,20.0,5.845,5.0,2.397,1303.0,88.941,1300.0,5.933,1304.0,2.551,2004.0,78.435,2002.0,6.574,2012.0,4.152,503.0,88.388,502.0,3.615,500.0,2.312
4,s3344-93,(LATEST SUMMARY) National Science Foundation A...,17.0,57.952,8.0,16.751,20.0,3.939,1798.0,83.761,1700.0,4.747,1709.0,3.163,898.0,41.903,800.0,25.726,803.0,19.244,2002.0,27.185,2000.0,19.686,2008.0,9.965


In [46]:
congress_level_df = pd.concat([congress_level_df, subject_df], axis=1)

### Status Sub-DF 

In [31]:
referred = []
reported = []

failed_in_suspension = []
cloture_failed = []

failed_origin_house = []
failed_origin_senate = []

passover_house = []
passover_senate = []

failed_second_house = []
failed_second_senate = []
passed = []

passback_house = []
passback_senate = []

passed_conference_house = []
passed_conference_senate = []

pocket_veto = []
prov_kill_veto = []

veto_override_fail_house = []
veto_override_fail_senate = []

veto_override_1_pass_house = []
veto_override_1_pass_senate = []

veto_override_2_fail_house = []
veto_override_2_fail_senate = []

enacted_veto_override = []
enacted_ten_day_rule = []
enacted_signed = []

list_of_statuses = [referred, reported, failed_in_suspension, cloture_failed,
                   failed_origin_house, failed_origin_senate, passover_house,
                   passover_senate, passed, passback_house, passback_senate,
                   passed_conference_house, passed_conference_senate, pocket_veto,
                   prov_kill_veto, veto_override_fail_house, veto_override_fail_senate,
                   veto_override_1_pass_house, veto_override_1_pass_senate,
                   veto_override_2_fail_house, veto_override_2_fail_senate,
                   enacted_veto_override, enacted_ten_day_rule, enacted_signed] 

print ('possible statuses:', len(list_of_statuses))

possible statuses: 24


In [29]:
def action_check(output_list, input_list, d, error_var, status):
    if error_var == 0:
        status_val = 0
        for i in range(len(input_list)): 
            if 'status' in input_list[i]:
                if (input_list[i]['status']) == str(status):
                    status_val +=1 
                    pass 
            else:
                pass
        output_list.append(status_val)
    else:
        output_list.append('feature_error') 

# Comprehensive Vote Detail Compiler

In [357]:
vote_status_df = status_detail_compiler(df, vote_status_dict) 

In [358]:
hopeful.head()

,cloture_failed,cloture_failed_roll,cloture_failed_vote_type,cloture_failed_acted_at,cloture_failed_how,cloture_failed_suspension,cloture_failed_result,fail_origin_h,fail_origin_h_roll,fail_origin_h_vote_type,fail_origin_h_acted_at,fail_origin_h_how,fail_origin_h_suspension,fail_origin_h_result,fail_origin_s,fail_origin_s_roll,fail_origin_s_vote_type,fail_origin_s_acted_at,fail_origin_s_how,fail_origin_s_suspension,fail_origin_s_result,passover_h,passover_h_roll,passover_h_vote_type,passover_h_acted_at,passover_h_how,passover_h_suspension,passover_h_result,passover_s,passover_s_roll,passover_s_vote_type,passover_s_acted_at,passover_s_how,passover_s_suspension,passover_s_result,fail_2nd_h,fail_2nd_h_roll,fail_2nd_h_vote_type,fail_2nd_h_acted_at,fail_2nd_h_how,fail_2nd_h_suspension,fail_2nd_h_result,fail_2nd_s,fail_2nd_s_roll,fail_2nd_s_vote_type,fail_2nd_s_acted_at,fail_2nd_s_how,fail_2nd_s_suspension,fail_2nd_s_result,passback_h,passback_h_roll,passback_h_vote_type,passback_h_acted_at,passback_h_how,passback_h_suspension,passback_h_result,passback_s,passback_s_roll,passback_s_vote_type,passback_s_acted_at,passback_s_how,passback_s_suspension,passback_s_result,fail_pingpong,fail_pingpong_roll,fail_pingpong_vote_type,fail_pingpong_acted_at,fail_pingpong_how,fail_pingpong_suspension,fail_pingpong_result,passed_bill,passed_bill_roll,passed_bill_vote_type,passed_bill_acted_at,passed_bill_how,passed_bill_suspension,passed_bill_result,conf_pass_h,conf_pass_h_roll,conf_pass_h_vote_type,conf_pass_h_acted_at,conf_pass_h_how,conf_pass_h_suspension,conf_pass_h_result,conf_pass_s,conf_pass_s_roll,conf_pass_s_vote_type,conf_pass_s_acted_at,conf_pass_s_how,conf_pass_s_suspension,conf_pass_s_result,fail_vo_1_h,fail_vo_1_h_roll,fail_vo_1_h_vote_type,fail_vo_1_h_acted_at,fail_vo_1_h_how,fail_vo_1_h_suspension,fail_vo_1_h_result,fail_vo_1_s,fail_vo_1_s_roll,fail_vo_1_s_vote_type,fail_vo_1_s_acted_at,fail_vo_1_s_how,fail_vo_1_s_suspension,fail_vo_1_s_result,vo_passover_h,vo_passover_h_roll,vo_passover_h_vote_type,vo_passover_h_acted_at,vo_passover_h_how,vo_passover_h_suspension,vo_passover_h_result,vo_passover_s,vo_passover_s_roll,vo_passover_s_vote_type,vo_passover_s_acted_at,vo_passover_s_how,vo_passover_s_suspension,vo_passover_s_result,fail_vo_2_h,fail_vo_2_h_roll,fail_vo_2_h_vote_type,fail_vo_2_h_acted_at,fail_vo_2_h_how,fail_vo_2_h_suspension,fail_vo_2_h_result,fail_vo_2_s,fail_vo_2_s_roll,fail_vo_2_s_vote_type,fail_vo_2_s_acted_at,fail_vo_2_s_how,fail_vo_2_s_suspension,fail_vo_2_s_result,enacted_vo,enacted_vo_roll,enacted_vo_vote_type,enacted_vo_acted_at,enacted_vo_how,enacted_vo_suspension,enacted_vo_result
0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN

In [360]:
vote_status_df = hopeful 

In [359]:
len(hopeful)

255449

In [365]:
referred = []
reported = []
enacted_signed = []
prov_kill_veto = []
vetoed_pocket = []
enacted_ten_day_rule = []

In [366]:
non_vote_statuses = ['REFERRED', 'REPORTED', 'ENACTED:SIGNED', 'PROV_KILL:VETO',
                     'VETOED:POCKET', 'ENACTED:TENDAYRULE']
non_vote_status_lists = [referred, reported, enacted_signed, prov_kill_veto, vetoed_pocket,
                        enacted_ten_day_rule] 

In [367]:
for i in range(len(non_vote_statuses)):
    action_compile_and_report(non_vote_statuses[i], non_vote_status_lists[i]) 

Positive Status Detected
1.0
Positive Status Detected
1.0
Positive Status Detected
1.0
Positive Status Detected
1.0
Positive Status Detected
1.0
Positive Status Detected
1.0


In [368]:
non_vote_df = pd.DataFrame(non_vote_status_lists).T 

In [370]:
non_vote_cols = ['referred', 'reported', 'enacted_signed', 'prov_kill_veto',
                 'vetoed_pocket', 'enacted_10d_rule']

In [371]:
non_vote_df.columns = non_vote_cols

In [372]:
non_vote_df.head()

,referred,reported,enacted_signed,prov_kill_veto,vetoed_pocket,enacted_10d_rule
0,1,0,0,0,0,0
1,0,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


In [373]:
status_detail_df = pd.concat([non_vote_df, vote_status_df], axis=1)

In [374]:
status_detail_df

,referred,reported,enacted_signed,prov_kill_veto,vetoed_pocket,enacted_10d_rule,cloture_failed,cloture_failed_roll,cloture_failed_vote_type,cloture_failed_acted_at,cloture_failed_how,cloture_failed_suspension,cloture_failed_result,fail_origin_h,fail_origin_h_roll,fail_origin_h_vote_type,fail_origin_h_acted_at,fail_origin_h_how,fail_origin_h_suspension,fail_origin_h_result,fail_origin_s,fail_origin_s_roll,fail_origin_s_vote_type,fail_origin_s_acted_at,fail_origin_s_how,fail_origin_s_suspension,fail_origin_s_result,passover_h,passover_h_roll,passover_h_vote_type,passover_h_acted_at,passover_h_how,passover_h_suspension,passover_h_result,passover_s,passover_s_roll,passover_s_vote_type,passover_s_acted_at,passover_s_how,passover_s_suspension,passover_s_result,fail_2nd_h,fail_2nd_h_roll,fail_2nd_h_vote_type,fail_2nd_h_acted_at,fail_2nd_h_how,fail_2nd_h_suspension,fail_2nd_h_result,fail_2nd_s,fail_2nd_s_roll,fail_2nd_s_vote_type,fail_2nd_s_acted_at,fail_2nd_s_how,fail_2nd_s_suspension,fail_2nd_s_result,passback_h,passback_h_roll,passback_h_vote_type,passback_h_acted_at,passback_h_how,passback_h_suspension,passback_h_result,passback_s,passback_s_roll,passback_s_vote_type,passback_s_acted_at,passback_s_how,passback_s_suspension,passback_s_result,fail_pingpong,fail_pingpong_roll,fail_pingpong_vote_type,fail_pingpong_acted_at,fail_pingpong_how,fail_pingpong_suspension,fail_pingpong_result,passed_bill,passed_bill_roll,passed_bill_vote_type,passed_bill_acted_at,passed_bill_how,passed_bill_suspension,passed_bill_result,conf_pass_h,conf_pass_h_roll,conf_pass_h_vote_type,conf_pass_h_acted_at,conf_pass_h_how,conf_pass_h_suspension,conf_pass_h_result,conf_pass_s,conf_pass_s_roll,conf_pass_s_vote_type,conf_pass_s_acted_at,conf_pass_s_how,conf_pass_s_suspension,conf_pass_s_result,fail_vo_1_h,fail_vo_1_h_roll,fail_vo_1_h_vote_type,fail_vo_1_h_acted_at,fail_vo_1_h_how,fail_vo_1_h_suspension,fail_vo_1_h_result,fail_vo_1_s,fail_vo_1_s_roll,fail_vo_1_s_vote_type,fail_vo_1_s_acted_at,fail_vo_1_s_how,fail_vo_1_s_suspension,fail_vo_1_s_result,vo_passover_h,vo_passover_h_roll,vo_passover_h_vote_type,vo_passover_h_acted_at,vo_passover_h_how,vo_passover_h_suspension,vo_passover_h_result,vo_passover_s,vo_passover_s_roll,vo_passover_s_vote_type,vo_passover_s_acted_at,vo_passover_s_how,vo_passover_s_suspension,vo_passover_s_result,fail_vo_2_h,fail_vo_2_h_roll,fail_vo_2_h_vote_type,fail_vo_2_h_acted_at,fail_vo_2_h_how,fail_vo_2_h_suspension,fail_vo_2_h_result,fail_vo_2_s,fail_vo_2_s_roll,fail_vo_2_s_vote_type,fail_vo_2_s_acted_at,fail_vo_2_s_how,fail_vo_2_s_suspension,fail_vo_2_s_result,enacted_vo,enacted_vo_roll,enacted_vo_vote_type,enacted_vo_acted_at,enacted_vo_how,enacted_vo_suspension,enacted_vo_result
0,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,

In [375]:
old_df = pd.read_csv('data_store/dataframes/full_congress_level_df2.csv')

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (23,24,25,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [376]:
kept_cols = list(old_df.columns)

In [377]:
kept_cols

['bill_id',
 'bill_type',
 'congress',
 'number',
 'action_count',
 'r_sponsor',
 'amendment_count',
 'committee_count',
 'cosponsor_count',
 'r_cosponsors',
 'd_cosponsors',
 'popular_title',
 'related_bill_count',
 'short_title',
 'subject_count',
 'title_count',
 'house_republicans',
 'house_democrats',
 'senate_republicans',
 'senate_democrats',
 'referral_committee_1',
 'referral_committee_2',
 'referral_committee_3',
 'markup_committee_1',
 'markup_committee_2',
 'markup_committee_3',
 'reporting_committee_1',
 'reporting_committee_2',
 'reporting_committee_3',
 'hearings_committee_1',
 'hearings_committee_2',
 'hearings_committee_3',
 'status',
 'bill_number',
 'summary',
 'major_label 1',
 'major_prob 1',
 'major_label 2',
 'major_prob 2',
 'major_label 3',
 'major_prob 3',
 'major_1_minor_label 1',
 'major_1_minor_prob 1',
 'major_1_minor_label 2',
 'major_1_minor_prob 2',
 'major_1_minor_label 3',
 'major_1_minor_prob 3',
 'major_2_minor_label 1',
 'major_2_minor_prob 1',
 'm

In [378]:
df0 = old_df[['bill_id',
 'bill_type',
 'congress',
 'number',
 'action_count',
 'r_sponsor',
 'amendment_count',
 'committee_count',
 'cosponsor_count',
 'r_cosponsors',
 'd_cosponsors',
 'popular_title',
 'related_bill_count',
 'short_title',
 'subject_count',
 'title_count',
 'house_republicans',
 'house_democrats',
 'senate_republicans',
 'senate_democrats',
 'referral_committee_1',
 'referral_committee_2',
 'referral_committee_3',
 'markup_committee_1',
 'markup_committee_2',
 'markup_committee_3',
 'reporting_committee_1',
 'reporting_committee_2',
 'reporting_committee_3',
 'hearings_committee_1',
 'hearings_committee_2',
 'hearings_committee_3',
 'status',
 'bill_number',
 'summary',
 'major_label 1',
 'major_prob 1',
 'major_label 2',
 'major_prob 2',
 'major_label 3',
 'major_prob 3',
 'major_1_minor_label 1',
 'major_1_minor_prob 1',
 'major_1_minor_label 2',
 'major_1_minor_prob 2',
 'major_1_minor_label 3',
 'major_1_minor_prob 3',
 'major_2_minor_label 1',
 'major_2_minor_prob 1',
 'major_2_minor_label 2',
 'major_2_minor_prob 2',
 'major_2_minor_label 3',
 'major_2_minor_prob 3',
 'major_3_minor_label 1',
 'major_3_minor_prob 1',
 'major_3_minor_label 2',
 'major_3_minor_prob 2',
 'major_3_minor_label 3',
 'major_3_minor_prob 3']]

In [379]:
df0.columns

Index(['bill_id', 'bill_type', 'congress', 'number', 'action_count',
       'r_sponsor', 'amendment_count', 'committee_count', 'cosponsor_count',
       'r_cosponsors', 'd_cosponsors', 'popular_title', 'related_bill_count',
       'short_title', 'subject_count', 'title_count', 'house_republicans',
       'house_democrats', 'senate_republicans', 'senate_democrats',
       'referral_committee_1', 'referral_committee_2', 'referral_committee_3',
       'markup_committee_1', 'markup_committee_2', 'markup_committee_3',
       'reporting_committee_1', 'reporting_committee_2',
       'reporting_committee_3', 'hearings_committee_1', 'hearings_committee_2',
       'hearings_committee_3', 'status', 'bill_number', 'summary',
       'major_label 1', 'major_prob 1', 'major_label 2', 'major_prob 2',
       'major_label 3', 'major_prob 3', 'major_1_minor_label 1',
       'major_1_minor_prob 1', 'major_1_minor_label 2', 'major_1_minor_prob 2',
       'major_1_minor_label 3', 'major_1_minor_prob 3',
   

In [380]:
full_congress_level_df3 = pd.concat([df0, status_detail_df], axis=1) 

In [382]:
full_congress_level_df3.shape 

(255449, 205)

In [383]:
full_congress_level_df3.to_csv('data_store/dataframes/full_congress_level_df3.csv') 

In [93]:
def status_detail_compiler(file_df, vote_dict):
    output_df = pd.DataFrame()
    for stat in range(len(vote_dict['vs'])):
        string_root = vote_dict['vs_roots'][stat]
        status = vote_dict['vs'][stat]
        s1 = string_root
        s2 = string_root + '_roll'
        s3 = string_root + '_vote_type'
        s4 = string_root + '_acted_at'
        s5 = string_root + '_how'
        s6 = string_root + '_suspension'
        s7 = string_root + '_result'
        col_list = [s1, s2, s3, s4, s5, s6, s7]
        status_list = []
        roll_list = []
        vote_type_list = []
        acted_at_list = []
        how_list = []
        suspension_list = []
        result_list = []
        for congress_number in file_df['congress'].unique():
            with zipfile.ZipFile('bill_metadata/' + str(congress_number) + '.zip', 'r') as z:
                for file in file_df['file_name'][
                    file_df['congress'] == congress_number].reset_index(drop=True):
                    with z.open(file) as f:
                        data = f.read()
                        d = json.loads(data.decode('utf-8'))
                        input_list = d['actions']
                        if 'actions' in d.keys():
                            idx_list = []
                            status_val = 0
                            roll_val = None
                            vote_type_val = None
                            acted_at_val = None
                            how_val = None
                            suspension_val = None
                            result_val = None
                            for i in range(len(input_list)):
                                if 'status' in input_list[i]:
                                    if input_list[i]['status'] == str(status):
                                        status_val +=1
                                        idx_list.append(i)
                                    else:
                                        pass 
                                else:
                                    pass
                            if len(idx_list) > 0:
                                idx = idx_list[-1]
                                status_list.append(status_val)
                                if 'roll' in input_list[idx]:
                                    roll_list.append(input_list[idx]['roll'])
                                else:
                                    roll_list.append(roll_val)
                                if 'vote_type' in input_list[idx]:
                                    vote_type_list.append(input_list[idx]['vote_type'])
                                else:
                                    vote_type_list.append(vote_type_val)
                                if 'acted_at' in input_list[idx]:
                                    acted_at_list.append(input_list[idx]['acted_at'])
                                else:
                                    acted_at_list.append(acted_at_val)
                                if 'how' in input_list[idx]:
                                    how_list.append(input_list[idx]['how'])
                                else:
                                    how_list.append(how_val)
                                positive_suspension = None 
                                for idx_val in idx_list: 
                                    if 'suspension' in input_list[idx_val]:
                                        if input_list[idx_val]['suspension'] == True:
                                            positive_suspension = True 
                                        else:
                                            pass
                                    else:
                                        pass
                                    suspension_list.append(positive_suspension) 
                                else:
                                    suspension_list.append(suspension_val)
                                if 'result' in input_list[idx]:
                                    result_list.append(input_list[idx]['result'])
                                else:
                                    result_list.append(result_val)
                            else:
                                status_list.append(status_val)
                                roll_list.append(roll_val)
                                vote_type_list.append(vote_type_val)
                                acted_at_list.append(acted_at_val)
                                how_list.append(how_val)
                                suspension_list.append(suspension_val)
                                result_list.append(result_val)
                        else:
                            'Something broke.'
        pre_df_list = [status_list, roll_list, vote_type_list, acted_at_list, how_list,
                      suspension_list, result_list]
        status_df = pd.DataFrame(pre_df_list).T
        status_df.columns = col_list
        #if len(vote_dict['vs'])
        if stat == 0:
            output_df= status_df
        else:
            output_df = pd.concat([output_df, status_df], axis=1)
    return output_df

In [31]:
def action_compile_and_report(status, output_list):
    status_compiler(file_df=df,
                    bill_metadata_name = 'actions',
                    output_list = output_list,
                    process_function = action_check,
                    status = status)
    
    if 1 in output_list:
        print ('Positive Status Detected')
    else:
        print ('No Positive Status Detected')
    
    print (len(output_list) / len(df))

In [286]:
def detail_report(status, status_detail_df):
    col_list = list(status_detail_df.columns)
    roll_list = [x for x in col_list[1] if x is not None]
    vote_type_list = [x for x in col_list[2] if x is not None]
    acted_at_list = [x for x in col_list[3] if x is not None]
    how_list = [x for x in col_list[4] if x is not None]
    suspension_list = [x for x in col_list[5] if x is not None]
    result_list = [x for x in col_list[6] if x is not None]
    if len(roll_list) > 0:
        print ('Positive ', status, ' Roll Number Status Detected')
    else:
        print ('No Positive ', status,  ' Roll Number Status Detected')
    if len(vote_type_list) > 0:
        print ('Positive ', status, ' Vote Type Status Detected')
    else:
        print ('No Positive ', status,  'Vote Type Status Detected')
    if len(acted_at_list) > 0:
        print ('Positive ', status, ' acted at Status Detected')
    else:
        print ('No Positive ', status,  ' acted at Status Detected')
    if len(how_list) > 0:
        print ('Positive ', status, ' How Status Detected')
    else:
        print ('No Positive ', status,  ' How Status Detected')
    if len(suspension_list) > 0:
        print ('Positive ', status, ' Suspension Status Detected')
    else:
        print ('No Positive ', status,  ' Suspension Status Detected')
    if len(result_list) > 0:
        print ('Positive ', status, ' Result Status Detected')
    else:
        print ('No Positive ', status,  ' Result Status Detected')
    print ('\n') 
    print ('Rolls:', len(roll_list) / len(status_detail_df))
    print ('Vote type:', len(vote_type_list) / len(status_detail_df))
    print ('Acted at:', len(acted_at_list) / len(status_detail_df))
    print ('How:', len(how_list) / len(status_detail_df))
    print ('Suspension:', len(suspension_list) / len(status_detail_df))
    print ('Result:', len(result_list) / len(status_detail_df))

In [32]:
vote_status_list = ['PROV_KILL:SUSPENSIONFAILED', 'PROV_KILL:CLOTUREFAILED',
                    'FAIL:ORIGINATING:HOUSE', 'FAIL:ORIGINATING:SENATE', 'PASS_OVER:HOUSE',
                   'PASS_OVER:SENATE', 'FAIL:SECOND:HOUSE', 'FAIL:SECOND:SENATE',
                   'PASS_BACK:HOUSE', 'PASS_BACK:SENATE', 'PROV_KILL:PINGPONGFAIL',
                   'PASSED:BILL', 'CONFERENCE:PASSED:HOUSE', 'CONFERENCE:PASSED:SENATE',
                   'VETOED:OVERRIDE_FAIL_ORIGINATING:HOUSE',
                    'VETOED:OVERRIDE_FAIL_ORIGINATING:SENATE',
                    'VETOED:OVERRIDE_PASS_OVER:HOUSE', 'VETOED:OVERRIDE_PASS_OVER:SENATE',
                   'VETOED:OVERRIDE_FAIL_SECOND:HOUSE', 'VETOED:OVERRIDE_FAIL_SECOND:SENATE',
                   'ENACTED:VETO_OVERRIDE']
vote_status_name_root_list = ['suspension_failed', 'cloture_failed', 'fail_origin_h',
                              'fail_origin_s', 'passover_h', 'passover_s', 'fail_2nd_h',
                              'fail_2nd_s', 'passback_h', 'passback_s', 'fail_pingpong',
                              'passed_bill', 'conf_pass_h', 'conf_pass_s', 'fail_vo_1_h',
                              'fail_vo_1_s', 'vo_passover_h', 'vo_passover_s',
                              'fail_vo_2_h', 'fail_vo_2_s', 'enacted_vo']

non_vote_status_list = ['REPORTED', 'REFERRED', 'ENACTED:SIGNED', 'PROV_KILL:VETO',
                        'VETOED:POCKET', 'ENACTED:TENDAYRULE']

In [28]:
susp_fail_status_list = ['PROV_KILL:SUSPENSIONFAILED']
susp_fail_root_list = ['suspension_failed']
susp_fail_dict = {'vs': susp_fail_status_list, 'vs_roots': susp_fail_root_list}

In [35]:
susp_fail_df = status_detail_compiler(df, susp_fail_dict) 

In [74]:
adjusted_status_df.head(1)

,suspension_failed,suspension_failed_roll,suspension_failed_vote_type,suspension_failed_acted_at,suspension_failed_how,suspension_failed_suspension,suspension_failed_result,cloture_failed,cloture_failed_roll,cloture_failed_vote_type,cloture_failed_acted_at,cloture_failed_how,cloture_failed_suspension,cloture_failed_result,fail_origin_h,fail_origin_h_roll,fail_origin_h_vote_type,fail_origin_h_acted_at,fail_origin_h_how,fail_origin_h_suspension,fail_origin_h_result,fail_origin_s,fail_origin_s_roll,fail_origin_s_vote_type,fail_origin_s_acted_at,fail_origin_s_how,fail_origin_s_suspension,fail_origin_s_result,passover_h,passover_h_roll,passover_h_vote_type,passover_h_acted_at,passover_h_how,passover_h_suspension,passover_h_result,passover_s,passover_s_roll,passover_s_vote_type,passover_s_acted_at,passover_s_how,passover_s_suspension,passover_s_result,fail_2nd_h,fail_2nd_h_roll,fail_2nd_h_vote_type,fail_2nd_h_acted_at,fail_2nd_h_how,fail_2nd_h_suspension,fail_2nd_h_result,fail_2nd_s,fail_2nd_s_roll,fail_2nd_s_vote_type,fail_2nd_s_acted_at,fail_2nd_s_how,fail_2nd_s_suspension,fail_2nd_s_result,passback_h,passback_h_roll,passback_h_vote_type,passback_h_acted_at,passback_h_how,passback_h_suspension,passback_h_result,passback_s,passback_s_roll,passback_s_vote_type,passback_s_acted_at,passback_s_how,passback_s_suspension,passback_s_result,fail_pingpong,fail_pingpong_roll,fail_pingpong_vote_type,fail_pingpong_acted_at,fail_pingpong_how,fail_pingpong_suspension,fail_pingpong_result,passed_bill,passed_bill_roll,passed_bill_vote_type,passed_bill_acted_at,passed_bill_how,passed_bill_suspension,passed_bill_result,conf_pass_h,conf_pass_h_roll,conf_pass_h_vote_type,conf_pass_h_acted_at,conf_pass_h_how,conf_pass_h_suspension,conf_pass_h_result,conf_pass_s,conf_pass_s_roll,conf_pass_s_vote_type,conf_pass_s_acted_at,conf_pass_s_how,conf_pass_s_suspension,conf_pass_s_result,fail_vo_1_h,fail_vo_1_h_roll,fail_vo_1_h_vote_type,fail_vo_1_h_acted_at,fail_vo_1_h_how,fail_vo_1_h_suspension,fail_vo_1_h_result,fail_vo_1_s,fail_vo_1_s_roll,fail_vo_1_s_vote_type,fail_vo_1_s_acted_at,fail_vo_1_s_how,fail_vo_1_s_suspension,fail_vo_1_s_result,vo_passover_h,vo_passover_h_roll,vo_passover_h_vote_type,vo_passover_h_acted_at,vo_passover_h_how,vo_passover_h_suspension,vo_passover_h_result,vo_passover_s,vo_passover_s_roll,vo_passover_s_vote_type,vo_passover_s_acted_at,vo_passover_s_how,vo_passover_s_suspension,vo_passover_s_result,fail_vo_2_h,fail_vo_2_h_roll,fail_vo_2_h_vote_type,fail_vo_2_h_acted_at,fail_vo_2_h_how,fail_vo_2_h_suspension,fail_vo_2_h_result,fail_vo_2_s,fail_vo_2_s_roll,fail_vo_2_s_vote_type,fail_vo_2_s_acted_at,fail_vo_2_s_how,fail_vo_2_s_suspension,fail_vo_2_s_result,enacted_vo,enacted_vo_roll,enacted_vo_vote_type,enacted_vo_acted_at,enacted_vo_how,enacted_vo_suspension,enacted_vo_result
0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
vote_related_statuses = list(adjusted_status_df.columns)

In [99]:
vote_status_suspension = vote_related_statuses[5::7] 
def unique_val_check(df, col_list):
    unique_vals = []
    positive_statuses = []
    output_dict = {'vals': unique_vals, 'cols': positive_statuses}
    for col in col_list:
        unique_to_col = list(df[col].unique())
        for val in unique_to_col:
            if val not in unique_vals:
                unique_vals.append(val)
                if val in [True, np.nan]:
                    positive_statuses.append(col)
                else:
                    pass
            else:
                pass
    
    return output_dict

In [100]:
suspension_check = unique_val_check(adjusted_status_df, vote_status_suspension)

In [101]:
suspension_check

{'vals': [nan, True, None],
 'cols': ['suspension_failed_suspension', 'suspension_failed_suspension']}

In [94]:
adjusted_status_df = status_detail_compiler(df, vote_status_dict) 

In [90]:
new_status_df = adjusted_status_df 

In [102]:
new_status_df.head(3)

,suspension_failed,suspension_failed_roll,suspension_failed_vote_type,suspension_failed_acted_at,suspension_failed_how,suspension_failed_suspension,suspension_failed_result,cloture_failed,cloture_failed_roll,cloture_failed_vote_type,cloture_failed_acted_at,cloture_failed_how,cloture_failed_suspension,cloture_failed_result,fail_origin_h,fail_origin_h_roll,fail_origin_h_vote_type,fail_origin_h_acted_at,fail_origin_h_how,fail_origin_h_suspension,fail_origin_h_result,fail_origin_s,fail_origin_s_roll,fail_origin_s_vote_type,fail_origin_s_acted_at,fail_origin_s_how,fail_origin_s_suspension,fail_origin_s_result,passover_h,passover_h_roll,passover_h_vote_type,passover_h_acted_at,passover_h_how,passover_h_suspension,passover_h_result,passover_s,passover_s_roll,passover_s_vote_type,passover_s_acted_at,passover_s_how,passover_s_suspension,passover_s_result,fail_2nd_h,fail_2nd_h_roll,fail_2nd_h_vote_type,fail_2nd_h_acted_at,fail_2nd_h_how,fail_2nd_h_suspension,fail_2nd_h_result,fail_2nd_s,fail_2nd_s_roll,fail_2nd_s_vote_type,fail_2nd_s_acted_at,fail_2nd_s_how,fail_2nd_s_suspension,fail_2nd_s_result,passback_h,passback_h_roll,passback_h_vote_type,passback_h_acted_at,passback_h_how,passback_h_suspension,passback_h_result,passback_s,passback_s_roll,passback_s_vote_type,passback_s_acted_at,passback_s_how,passback_s_suspension,passback_s_result,fail_pingpong,fail_pingpong_roll,fail_pingpong_vote_type,fail_pingpong_acted_at,fail_pingpong_how,fail_pingpong_suspension,fail_pingpong_result,passed_bill,passed_bill_roll,passed_bill_vote_type,passed_bill_acted_at,passed_bill_how,passed_bill_suspension,passed_bill_result,conf_pass_h,conf_pass_h_roll,conf_pass_h_vote_type,conf_pass_h_acted_at,conf_pass_h_how,conf_pass_h_suspension,conf_pass_h_result,conf_pass_s,conf_pass_s_roll,conf_pass_s_vote_type,conf_pass_s_acted_at,conf_pass_s_how,conf_pass_s_suspension,conf_pass_s_result,fail_vo_1_h,fail_vo_1_h_roll,fail_vo_1_h_vote_type,fail_vo_1_h_acted_at,fail_vo_1_h_how,fail_vo_1_h_suspension,fail_vo_1_h_result,fail_vo_1_s,fail_vo_1_s_roll,fail_vo_1_s_vote_type,fail_vo_1_s_acted_at,fail_vo_1_s_how,fail_vo_1_s_suspension,fail_vo_1_s_result,vo_passover_h,vo_passover_h_roll,vo_passover_h_vote_type,vo_passover_h_acted_at,vo_passover_h_how,vo_passover_h_suspension,vo_passover_h_result,vo_passover_s,vo_passover_s_roll,vo_passover_s_vote_type,vo_passover_s_acted_at,vo_passover_s_how,vo_passover_s_suspension,vo_passover_s_result,fail_vo_2_h,fail_vo_2_h_roll,fail_vo_2_h_vote_type,fail_vo_2_h_acted_at,fail_vo_2_h_how,fail_vo_2_h_suspension,fail_vo_2_h_result,fail_vo_2_s,fail_vo_2_s_roll,fail_vo_2_s_vote_type,fail_vo_2_s_acted_at,fail_vo_2_s_how,fail_vo_2_s_suspension,fail_vo_2_s_result,enacted_vo,enacted_vo_roll,enacted_vo_vote_type,enacted_vo_acted_at,enacted_vo_how,enacted_vo_suspension,enacted_vo_result
0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN

In [103]:
new_status_df.shape 

(255449, 147)

In [104]:
new_status_df.to_csv('data_store/dataframes/full_congress_level_df4.csv', index=False)

In [116]:
other_features = pd.read_csv('data_store/dataframes/full_congress_level_df3.csv') 

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (23,24,25,26,29,30,31,32,68,69,70,72,75,76,77,79,82,83,84,86,89,90,91,92,93,96,97,98,100,103,104,105,107,110,111,112,114,117,118,119,120,121,124,125,126,128,131,132,133,135,138,139,140,141,142,145,146,147,148,149,152,153,154,156,159,160,161,163,166,167,168,170,173,174,175,177,180,181,182,184,187,188,189,191,194,195,196,198,201,202,203,205) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [117]:
keep_col_list = list(other_features.columns) 

In [120]:
keep_col_list = ['bill_id',
 'bill_type',
 'congress',
 'number',
 'action_count',
 'r_sponsor',
 'amendment_count',
 'committee_count',
 'cosponsor_count',
 'r_cosponsors',
 'd_cosponsors',
 'popular_title',
 'related_bill_count',
 'short_title',
 'subject_count',
 'title_count',
 'house_republicans',
 'house_democrats',
 'senate_republicans',
 'senate_democrats',
 'referral_committee_1',
 'referral_committee_2',
 'referral_committee_3',
 'markup_committee_1',
 'markup_committee_2',
 'markup_committee_3',
 'reporting_committee_1',
 'reporting_committee_2',
 'reporting_committee_3',
 'hearings_committee_1',
 'hearings_committee_2',
 'hearings_committee_3',
 'status',
 'bill_number',
 'summary',
 'major_label 1',
 'major_prob 1',
 'major_label 2',
 'major_prob 2',
 'major_label 3',
 'major_prob 3',
 'major_1_minor_label 1',
 'major_1_minor_prob 1',
 'major_1_minor_label 2',
 'major_1_minor_prob 2',
 'major_1_minor_label 3',
 'major_1_minor_prob 3',
 'major_2_minor_label 1',
 'major_2_minor_prob 1',
 'major_2_minor_label 2',
 'major_2_minor_prob 2',
 'major_2_minor_label 3',
 'major_2_minor_prob 3',
 'major_3_minor_label 1',
 'major_3_minor_prob 1',
 'major_3_minor_label 2',
 'major_3_minor_prob 2',
 'major_3_minor_label 3',
 'major_3_minor_prob 3',
 'referred',
 'reported',
 'enacted_signed',
 'prov_kill_veto',
 'vetoed_pocket',
 'enacted_10d_rule']

In [121]:
other_features = other_features[keep_col_list]

In [122]:
other_features.columns

Index(['bill_id', 'bill_type', 'congress', 'number', 'action_count',
       'r_sponsor', 'amendment_count', 'committee_count', 'cosponsor_count',
       'r_cosponsors', 'd_cosponsors', 'popular_title', 'related_bill_count',
       'short_title', 'subject_count', 'title_count', 'house_republicans',
       'house_democrats', 'senate_republicans', 'senate_democrats',
       'referral_committee_1', 'referral_committee_2', 'referral_committee_3',
       'markup_committee_1', 'markup_committee_2', 'markup_committee_3',
       'reporting_committee_1', 'reporting_committee_2',
       'reporting_committee_3', 'hearings_committee_1', 'hearings_committee_2',
       'hearings_committee_3', 'status', 'bill_number', 'summary',
       'major_label 1', 'major_prob 1', 'major_label 2', 'major_prob 2',
       'major_label 3', 'major_prob 3', 'major_1_minor_label 1',
       'major_1_minor_prob 1', 'major_1_minor_label 2', 'major_1_minor_prob 2',
       'major_1_minor_label 3', 'major_1_minor_prob 3',
   

In [123]:
new_df = pd.concat([other_features, new_status_df], axis=1)

In [124]:
new_df.to_csv('data_store/dataframes/full_congress_level_df6.csv', index=False)

In [37]:
susp_fail_df.to_csv('data_store/dataframes/susp_fail_df.csv', index=)

,suspension_failed,suspension_failed_roll,suspension_failed_vote_type,suspension_failed_acted_at,suspension_failed_how,suspension_failed_suspension,suspension_failed_result
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,NaN,NaN,NaN,NaN
5,0,NaN,NaN,NaN,NaN,NaN,NaN
6,0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,NaN,NaN,NaN,NaN,NaN,NaN
8,0,NaN,NaN,NaN,NaN,NaN,NaN
9,0,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
vote_status_dict = {'vs': vote_status_list, 'vs_roots': vote_status_name_root_list}

In [243]:
testing_things = ['cat', 'bear', 'another_animal']

In [244]:
testing_things[2] 

'another_animal'

In [242]:
vote_status_dict['vs'][7]

'FAIL:SECOND:SENATE'

In [229]:
non_vote_status_list

['REPORTED',
 'REFERRED',
 'ENACTED:SIGNED',
 'PROV_KILL:VETO',
 'VETOED:POCKET',
 'ENACTED:TENDAYRULE']

In [233]:
len(vote_status_list) 

21

In [234]:
len(vote_status_name_root_list) 

21

### Status: REFERRED

In [37]:
referred = []
compile_referred = action_compile_and_report(status='REFERRED',
                                             output_list= referred)

Positive Status Detected
1.0


### Status: REPORTED 

In [38]:
reported = []
compile_reported = action_compile_and_report(status='REPORTED',
                                             output_list= reported) 

Positive Status Detected
1.0


In [171]:
reported_referred_df = pd.DataFrame([referred, reported]).T 

In [172]:
reported_referred_df.columns = ['referred', 'reported']

In [173]:
reported_referred_df.head(3)

,referred,reported
0,1,0
1,0,0
2,1,0


### Status: PROV_KILL:SUSPENSIONFAILED

In [125]:
suspension_failed = []

suspension_failed_roll = []
suspension_failed_type = []
suspension_failed_at = []
suspension_failed_how = []
suspension_failed_suspension = []
suspension_failed_result = []

suspension_failed_details = [
    suspension_failed_roll, suspension_failed_type, suspension_failed_at,
    suspension_failed_how, suspension_failed_suspension, suspension_failed_result
] 

In [126]:
compile_suspension_failed = action_compile_and_report(status='PROV_KILL:SUSPENSIONFAILED',
                                             output_list= suspension_failed) 

Positive Status Detected
1.0


In [127]:
susp_failed_details = detail_compile_and_report(status='PROV_KILL:SUSPENSIONFAILED',
                                            output_list=suspension_failed_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.0007046416310104953
Vote type: 0.0007046416310104953
Acted at: 0.0007046416310104953
How: 0.0007046416310104953
Suspension: 0.0007046416310104953
Result: 0.0007046416310104953


In [176]:
def status_df_builder(previous_df, status_list, details_list, new_col_names):
    current_col_names = list(previous_df.columns)
    status_df = pd.DataFrame(status_list)
    details_df = pd.DataFrame(details_list).T
    new_df = pd.concat([status_df, details_df], axis=1)
    new_df_col_names = current_col_names.extend(new_col_names)
    df = pd.concat([previous_df, new_df], axis=1)
    df.columns = current_col_names
    return df  

 def column_lister(first_half, status_list_name):
    s1 = status_list_name
    s2 = first_half + '_roll'
    s3 = first_half + '_vote_type'
    s4 = first_half + '_acted_at'
    s5 = first_half + '_how'
    s6 = first_half + '_suspension'
    s7 = first_half + '_result'
    output_col_list = [s1, s2, s3, s4, s5, s6, s7]
    print (output_col_list)
    return output_col_list
       

In [132]:
len(suspension_failed_details[0])

255449

In [133]:
len(reported_referred_df)

255449

In [181]:
susp_failed_cols = ['suspension_failed', 'suspension_failed_rolls', 'suspension_failed_type',
                   'suspension_failed_at', 'suspension_failed_how',
                    'suspension_failed_suspension', 'suspension_failed_result']

In [174]:
test_list = list(reported_referred_df.columns)
test_col_list = test_list.extend(susp_failed_cols) 

In [180]:
test_list

['referred',
 'reported',
 'suspension_failed',
 'suspension_failed_rolls, suspension_failed_type',
 'suspension_failed_at',
 'suspension_failed_how',
 'suspension_failed_suspension',
 'suspension_failed_result']

In [168]:
test_col_list

In [182]:
df = status_df_builder(reported_referred_df, suspension_failed,
                       suspension_failed_details, susp_failed_cols) 

In [183]:
df 

,referred,reported,suspension_failed,suspension_failed_rolls,suspension_failed_type,suspension_failed_at,suspension_failed_how,suspension_failed_suspension,suspension_failed_result
0,1,0,0,None,None,None,None,None,None
1,0,0,0,None,None,None,None,None,None
2,1,0,0,None,None,None,None,None,None
3,1,0,0,None,None,None,None,None,None
4,1,0,0,None,None,None,None,None,None
5,1,0,0,None,None,None,None,None,None
6,1,0,0,None,None,None,None,None,None
7,1,0,0,None,None,None,None,None,None
8,1,0,0,None,None,None,None,None,None
9,1,0,0,None,None,None,None,None,None


In [196]:
def column_lister(first_half, status_list_name):
    s1 = status_list_name
    s2 = first_half + '_roll'
    s3 = first_half + '_vote_type'
    s4 = first_half + '_acted_at'
    s5 = first_half + '_how'
    s6 = first_half + '_suspension'
    s7 = first_half + '_result'
    output_col_list = [s1, s2, s3, s4, s5, s6, s7]
    print (output_col_list)
    return output_col_list
    

### Status: PROV_KILL:CLOTUREFAILED

In [46]:
cloture_failed = []

cloture_failed_roll = []
cloture_failed_type = []
cloture_failed_at = []
cloture_failed_how = []
cloture_failed_suspension = []
cloture_failed_result = []

cloture_failed_details = [
    cloture_failed_roll, cloture_failed_type, cloture_failed_at,
    cloture_failed_how, cloture_failed_suspension, cloture_failed_result
] 

In [47]:
compile_cloture_failed = action_compile_and_report(status='PROV_KILL:CLOTUREFAILED',
                                             output_list= cloture_failed) 

Positive Status Detected
1.0


In [48]:
cloture_failure_details = detail_compile_and_report(status='PROV_KILL:CLOTUREFAILED',
                                            output_list=cloture_failed_details)  

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.0004815051145238384
Vote type: 0.0004815051145238384
Acted at: 0.0004815051145238384
How: 0.0004815051145238384
Suspension: 0.0
Result: 0.0004815051145238384


In [186]:
cloture_failed_cols = ['cloture_failed', 'cloture_failed_roll', 'cloture_failed_type',
                       'cloture_failed_at', 'cloture_failed_how',
                       'cloture_failed_suspension', 'cloture_failed_result']

In [187]:
df = status_df_builder(df, cloture_failed,
                       cloture_failed_details, cloture_failed_cols) 

In [188]:
df.head(3)

,referred,reported,suspension_failed,suspension_failed_rolls,suspension_failed_type,suspension_failed_at,suspension_failed_how,suspension_failed_suspension,suspension_failed_result,cloture_failed,cloture_failed_roll,cloture_failed_type,cloture_failed_at,cloture_failed_how,cloture_failed_suspension,cloture_failed_result
0,1,0,0,None,None,None,None,None,None,0,None,None,None,None,None,None
1,0,0,0,None,None,None,None,None,None,0,None,None,None,None,None,None
2,1,0,0,None,None,None,None,None,None,0,None,None,None,None,None,None


### Status: FAIL:ORIGINATING:HOUSE

In [39]:
failed_origin_house = []

fail_origin_h_roll = []
fail_origin_h_vote_type = []
fail_origin_h_acted_at = []
fail_origin_h_how = []
fail_origin_h_suspension = []
fail_origin_h_result = []
fail_origin_h_details = [
    fail_origin_h_roll, fail_origin_h_vote_type, fail_origin_h_acted_at, fail_origin_h_how,
    fail_origin_h_suspension, fail_origin_h_result
]

In [40]:
compile_failed_origin_house = action_compile_and_report(status='FAIL:ORIGINATING:HOUSE',
                                                       output_list = failed_origin_house)

Positive Status Detected
1.0


In [41]:
failed_origin_house_details = detail_compile_and_report(status='FAIL:ORIGINATING:HOUSE',
                                            output_list=fail_origin_h_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.00011744027183508255
Vote type: 0.00012135494756291862
Acted at: 0.00012135494756291862
How: 0.00012135494756291862
Suspension: 0.0
Result: 0.00012135494756291862


In [197]:
f_o_h_cols = column_lister('failed_origin_h', 'failed_origin_house')

['failed_origin_house', 'failed_origin_h_roll', 'failed_origin_h_vote_type', 'failed_origin_h_acted_at', 'failed_origin_h_how', 'failed_origin_h_suspension', 'failed_origin_h_result']


In [198]:
df = status_df_builder(df, failed_origin_house,
                       fail_origin_h_details, f_o_h_cols) 

In [199]:
df.head(3) 

,referred,reported,suspension_failed,suspension_failed_rolls,suspension_failed_type,suspension_failed_at,suspension_failed_how,suspension_failed_suspension,suspension_failed_result,cloture_failed,cloture_failed_roll,cloture_failed_type,cloture_failed_at,cloture_failed_how,cloture_failed_suspension,cloture_failed_result,failed_origin_house,failed_origin_h_roll,failed_origin_h_vote_type,failed_origin_h_acted_at,failed_origin_h_how,failed_origin_h_suspension,failed_origin_h_result
0,1,0,0,None,None,None,None,None,None,0,None,None,None,None,None,None,0,None,None,None,None,None,None
1,0,0,0,None,None,None,None,None,None,0,None,None,None,None,None,None,0,None,None,None,None,None,None
2,1,0,0,None,None,None,None,None,None,0,None,None,None,None,None,None,0,None,None,None,None,None,None


### Status: FAIL:ORIGINATING:SENATE

In [49]:
failed_origin_senate = []

fail_origin_s_roll = []
fail_origin_s_vote_type = []
fail_origin_s_acted_at = [] 
fail_origin_s_how = []
fail_origin_s_suspension = []
fail_origin_s_result = []
fail_origin_s_details = [
    fail_origin_s_roll, fail_origin_s_vote_type, fail_origin_s_acted_at, fail_origin_s_how,
    fail_origin_s_suspension, fail_origin_s_result
]

In [50]:
compile_failed_origin_senate = action_compile_and_report(status='FAIL:ORIGINATING:SENATE',
                                                       output_list = failed_origin_senate)

Positive Status Detected
1.0


In [51]:
failed_origin_senate_details = detail_compile_and_report(status='FAIL:ORIGINATING:SENATE',
                                            output_list=fail_origin_s_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 2.3488054367016507e-05
Vote type: 2.3488054367016507e-05
Acted at: 2.3488054367016507e-05
How: 2.3488054367016507e-05
Suspension: 0.0
Result: 2.3488054367016507e-05


In [200]:
f_o_s_cols = column_lister('failed_origin_s', 'failed_origin_senate') 

['failed_origin_senate', 'failed_origin_s_roll', 'failed_origin_s_vote_type', 'failed_origin_s_acted_at', 'failed_origin_s_how', 'failed_origin_s_suspension', 'failed_origin_s_result']


In [201]:
df = status_df_builder(df, failed_origin_senate,
                       fail_origin_s_details, f_o_s_cols) 

### Status: PASS_OVER:HOUSE

In [52]:
passover_house = []

passover_h_roll = []
passover_h_vote_type = []
passover_h_acted_at = []
passover_h_how = []
passover_h_suspension = []
passover_h_result = [] 

passover_h_details = [
    passover_h_roll, passover_h_vote_type, passover_h_acted_at, passover_h_how,
    passover_h_suspension, passover_h_result
] 

In [202]:
p_o_h_cols = column_lister('passover_h', 'passover_house')

['passover_house', 'passover_h_roll', 'passover_h_vote_type', 'passover_h_acted_at', 'passover_h_how', 'passover_h_suspension', 'passover_h_result']


In [53]:
compile_passover_house = action_compile_and_report(status='PASS_OVER:HOUSE', 
                                                       output_list = passover_house)

Positive Status Detected
1.0


In [54]:
passover_house_details = detail_compile_and_report(status='PASS_OVER:HOUSE',
                                            output_list=passover_h_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.01724806125684579
Vote type: 0.04723839200779804
Acted at: 0.04723839200779804
How: 0.04723839200779804
Suspension: 0.007856754185767022
Result: 0.04723839200779804


In [203]:
df = status_df_builder(df, passover_house,
                       passover_h_details, p_o_h_cols) 

### Status: PASS_OVER:SENATE

In [55]:
passover_senate = []

passover_s_roll = []
passover_s_vote_type = []
passover_s_acted_at = []
passover_s_how = []
passover_s_suspension = []
passover_s_result = []

passover_s_details = [
    passover_s_roll, passover_s_vote_type, passover_s_acted_at, passover_s_how,
    passover_s_suspension, passover_s_result
] 

In [204]:
p_o_s_cols = column_lister('passover_s', 'passover_senate') 

['passover_senate', 'passover_s_roll', 'passover_s_vote_type', 'passover_s_acted_at', 'passover_s_how', 'passover_s_suspension', 'passover_s_result']


In [56]:
compile_passover_senate = action_compile_and_report(status='PASS_OVER:SENATE',
                                                       output_list = passover_senate)

Positive Status Detected
1.0


In [57]:
passover_senate_details = detail_compile_and_report(status='PASS_OVER:SENATE',
                                            output_list=passover_s_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.001526723533856073
Vote type: 0.020442436650760034
Acted at: 0.020442436650760034
How: 0.020442436650760034
Suspension: 0.0
Result: 0.020442436650760034


In [205]:
df = status_df_builder(df, passover_senate,
                       passover_s_details, p_o_s_cols) 

## Status: FAIL:SECOND:HOUSE

In [58]:
failed_second_house = []

failed_second_h_roll = []
failed_second_h_vote_type = []
failed_second_h_acted_at = []
failed_second_h_how = []
failed_second_h_suspension = []
failed_second_h_result = []

failed_second_h_details = [
    failed_second_h_roll, failed_second_h_vote_type, failed_second_h_acted_at,
    failed_second_h_how, failed_second_h_suspension, failed_second_h_result
] 

In [206]:
fail2_house_cols = column_lister('failed_second_h', 'failed_second_house') 

['failed_second_house', 'failed_second_h_roll', 'failed_second_h_vote_type', 'failed_second_h_acted_at', 'failed_second_h_how', 'failed_second_h_suspension', 'failed_second_h_result']


In [59]:
compile_fail2_house = action_compile_and_report(status='FAIL:SECOND:HOUSE',
                                                       output_list = failed_second_house) 

Positive Status Detected
1.0


In [60]:
fail2_house_details = detail_compile_and_report(status='FAIL:SECOND:HOUSE',
                                            output_list=failed_second_h_details) 

No Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.0
Vote type: 3.914675727836085e-06
Acted at: 3.914675727836085e-06
How: 3.914675727836085e-06
Suspension: 0.0
Result: 3.914675727836085e-06


In [207]:
df = status_df_builder(df, failed_second_house,
                       failed_second_h_details, fail2_house_cols) 

## Status: FAIL:SECOND:SENATE

In [61]:
failed_second_senate = []

failed_second_s_roll = []
failed_second_s_vote_type = []
failed_second_s_acted_at = []
failed_second_s_how = []
failed_second_s_suspension = []
failed_second_s_result = []

failed_second_s_details = [
    failed_second_s_roll, failed_second_s_vote_type, failed_second_s_acted_at,
    failed_second_s_how, failed_second_s_suspension, failed_second_s_result
] 

In [208]:
fail2_senate_cols = column_lister('failed_second_s', 'failed_second_senate') 

['failed_second_senate', 'failed_second_s_roll', 'failed_second_s_vote_type', 'failed_second_s_acted_at', 'failed_second_s_how', 'failed_second_s_suspension', 'failed_second_s_result']


In [62]:
compile_fail2_senate = action_compile_and_report(status='FAIL:SECOND:SENATE',
                                                       output_list = failed_second_senate)

Positive Status Detected
1.0


In [63]:
fail2_senate_details = detail_compile_and_report(status='FAIL:SECOND:SENATE',
                                            output_list=failed_second_s_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 1.565870291134434e-05
Vote type: 1.565870291134434e-05
Acted at: 1.565870291134434e-05
How: 1.565870291134434e-05
Suspension: 0.0
Result: 1.565870291134434e-05


In [209]:
df = status_df_builder(df, failed_second_senate,
                       failed_second_s_details, fail2_senate_cols) 

### Status: PASSED:BILL

In [64]:
passed = []

passed_roll = []
passed_vote_type = []
passed_acted_at = []
passed_how = []
passed_suspension = []
passed_result = []

passed_details = [
    passed_roll, passed_vote_type, passed_acted_at,
    passed_how, passed_suspension, passed_result
] 

In [210]:
passed_cols = column_lister('passed', 'passed') 

['passed', 'passed_roll', 'passed_vote_type', 'passed_acted_at', 'passed_how', 'passed_suspension', 'passed_result']


In [65]:
compile_passed_bill = action_compile_and_report(status='PASSED:BILL',
                                                       output_list = passed)

Positive Status Detected
1.0


In [66]:
passed_bill_details = detail_compile_and_report(status='PASSED:BILL',
                                            output_list=passed_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.003366621125939033
Vote type: 0.02744579152785879
Acted at: 0.02744579152785879
How: 0.02744579152785879
Suspension: 0.0008494846329404304
Result: 0.02744579152785879


In [211]:
df = status_df_builder(df, passed,
                       passed_details, passed_cols) 

In [212]:
df.shape 

(255449, 65)

### Status: PASS_BACK:HOUSE

In [67]:
passback_house = []

passback_h_roll = []
passback_h_vote_type = []
passback_h_acted_at = []
passback_h_how = []
passback_h_suspension = []
passback_h_result = [] 

passback_h_details = [
    passback_h_roll, passback_h_vote_type, passback_h_acted_at, passback_h_how,
    passback_h_suspension, passback_h_result
] 

In [213]:
passback_h_cols = column_lister('passback_h', 'passback_house') 

['passback_house', 'passback_h_roll', 'passback_h_vote_type', 'passback_h_acted_at', 'passback_h_how', 'passback_h_suspension', 'passback_h_result']


In [68]:
compile_passback_house = action_compile_and_report(status='PASS_BACK:HOUSE',
                                                       output_list = passback_house) 

Positive Status Detected
1.0


In [69]:
passback_house_details = detail_compile_and_report(status='PASS_BACK:HOUSE',
                                            output_list=passback_h_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.00040321159996711673
Vote type: 0.0024662457085367333
Acted at: 0.0024662457085367333
How: 0.0024662457085367333
Suspension: 0.00024270989512583725
Result: 0.0024662457085367333


In [214]:
df = status_df_builder(df, passback_house,
                       passback_h_details, passback_h_cols) 

### Status: PASS_BACK:SENATE

In [70]:
passback_senate = []

passback_s_roll = []
passback_s_vote_type = []
passback_s_acted_at = []
passback_s_how = []
passback_s_suspension = []
passback_s_result = [] 

passback_s_details = [
    passback_s_roll, passback_s_vote_type, passback_s_acted_at, passback_s_how,
    passback_s_suspension, passback_s_result
] 

In [215]:
passback_s_cols = column_lister('passback_s', 'passback_senate') 

['passback_senate', 'passback_s_roll', 'passback_s_vote_type', 'passback_s_acted_at', 'passback_s_how', 'passback_s_suspension', 'passback_s_result']


In [71]:
compile_passback_senate = action_compile_and_report(status='PASS_BACK:SENATE',
                                                       output_list = passback_senate)

Positive Status Detected
1.0


In [72]:
passback_senate_details = detail_compile_and_report(status='PASS_BACK:SENATE',
                                            output_list=passback_s_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.001894703052272665
Vote type: 0.008381320733297058
Acted at: 0.008381320733297058
How: 0.008381320733297058
Suspension: 0.0
Result: 0.008381320733297058


In [216]:
df = status_df_builder(df, passback_senate,
                       passback_s_details, passback_s_cols) 

### Status: CONFERENCE:PASSED:HOUSE

In [73]:
passed_conference_house = []

pass_conf_h_roll = []
pass_conf_h_vote_type = []
pass_conf_h_acted_at = []
pass_conf_h_how = []
pass_conf_h_suspension = []
pass_conf_h_result = [] 

pass_conf_h_details = [
    pass_conf_h_roll, pass_conf_h_vote_type, pass_conf_h_acted_at, pass_conf_h_how,
    pass_conf_h_suspension, pass_conf_h_result
] 

In [217]:
pass_conf_h_cols = column_lister('pass_conf_h', 'passed_conference_house') 

['passed_conference_house', 'pass_conf_h_roll', 'pass_conf_h_vote_type', 'pass_conf_h_acted_at', 'pass_conf_h_how', 'pass_conf_h_suspension', 'pass_conf_h_result']


In [74]:
compile_pass_conf_house = action_compile_and_report(status='CONFERENCE:PASSED:HOUSE',
                                                       output_list = passed_conference_house)

Positive Status Detected
1.0


In [75]:
passed_conf_h_details = detail_compile_and_report(status='CONFERENCE:PASSED:HOUSE',
                                            output_list=pass_conf_h_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.0013035870173694162
Vote type: 0.0016441638056911556
Acted at: 0.0016441638056911556
How: 0.0016441638056911556
Suspension: 7.82935145567217e-06
Result: 0.0016441638056911556


In [218]:
df = status_df_builder(df, passed_conference_house,
                       pass_conf_h_details, pass_conf_h_cols)  

### Status: CONFERENCE:PASSED:SENATE 

In [224]:
passed_conference_senate = []

pass_conf_s_roll = []
pass_conf_s_vote_type = []
pass_conf_s_acted_at = []
pass_conf_s_how = []
pass_conf_s_suspension = []
pass_conf_s_result = [] 

pass_conf_s_details = [
    pass_conf_s_roll, pass_conf_s_vote_type, pass_conf_s_acted_at, pass_conf_s_how,
    pass_conf_s_suspension, pass_conf_s_result
] 

In [225]:
pass_conf_s_cols = column_lister('pass_conf_s', 'passed_conference_senate') 

['passed_conference_senate', 'pass_conf_s_roll', 'pass_conf_s_vote_type', 'pass_conf_s_acted_at', 'pass_conf_s_how', 'pass_conf_s_suspension', 'pass_conf_s_result']


In [1]:
compile_pass_conf_senate = action_compile_and_report(status='CONFERENCE:PASSED:SENATE',
                                                       output_list = passed_conference_senate)

In [77]:
passed_conf_s_details = detail_compile_and_report(status='CONFERENCE:PASSED:SENATE',
                                            output_list=pass_conf_s_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 0.0003797235456001002
Vote type: 0.0013740511804704657
Acted at: 0.0013740511804704657
How: 0.0013740511804704657
Suspension: 0.0
Result: 0.0013740511804704657


In [223]:
df.shape 

(255449, 86)

In [2]:
df = status_df_builder(df, passed_conference_senate,
                       pass_conf_s_details, pass_conf_s_cols)  

### Status: VETOED:POCKET

In [78]:
pocket_veto = []
compile_pocket_veto = action_compile_and_report(status='VETOED:POCKET',
                                                       output_list = pocket_veto)

Positive Status Detected
1.0


### Status: PROV_KILL:VETO

In [80]:
prov_kill_veto = []
compile_prov_kill_veto = action_compile_and_report(status='PROV_KILL:VETO',
                                                       output_list = prov_kill_veto)

Positive Status Detected
1.0


### Status: VETOED:OVERRIDE_FAIL_ORIGINATING:HOUSE

In [81]:
veto_override_fail_house = []

vo_fail_h_roll = []
vo_fail_h_vote_type = []
vo_fail_h_acted_at = []
vo_fail_h_how = []
vo_fail_h_suspension = []
vo_fail_h_result = [] 

vo_fail_h_details = [
    vo_fail_h_roll, vo_fail_h_vote_type, vo_fail_h_acted_at, vo_fail_h_how,
    vo_fail_h_suspension, vo_fail_h_result
] 

In [294]:
compile_v_override_fail_house = action_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_ORIGINATING:HOUSE',
    output_list= veto_override_fail_house) 

Positive Status Detected
1.0


In [82]:
v_override_fail_h_details = detail_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_ORIGINATING:HOUSE',
    output_list= vo_fail_h_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 9.003754174022995e-05
Vote type: 9.003754174022995e-05
Acted at: 9.003754174022995e-05
How: 9.003754174022995e-05
Suspension: 0.0
Result: 9.003754174022995e-05


### Status: VETOED:OVERRIDE_FAIL_ORIGINATING:SENATE

In [83]:
veto_override_fail_senate = []

vo_fail_s_roll = []
vo_fail_s_vote_type = []
vo_fail_s_acted_at = []
vo_fail_s_how = []
vo_fail_s_suspension = []
vo_fail_s_result = [] 

vo_fail_s_details = [
    vo_fail_s_roll, vo_fail_s_vote_type, vo_fail_s_acted_at, vo_fail_s_how,
    vo_fail_s_suspension, vo_fail_s_result
] 

In [84]:
compile_v_override_fail_senate = action_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_ORIGINATING:SENATE',
    output_list= veto_override_fail_senate) 

Positive Status Detected
1.0


In [85]:
v_override_fail_s_details = detail_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_ORIGINATING:SENATE',
    output_list= vo_fail_s_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 2.7402730094852592e-05
Vote type: 2.7402730094852592e-05
Acted at: 2.7402730094852592e-05
How: 2.7402730094852592e-05
Suspension: 0.0
Result: 2.7402730094852592e-05


### Status: VETOED:OVERRIDE_PASS_OVER:HOUSE

In [86]:
veto_override_1_pass_house = []

vo_pass_1_h_roll = []
vo_pass_1_h_vote_type = []
vo_pass_1_h_acted_at = []
vo_pass_1_h_how = []
vo_pass_1_h_suspension = []
vo_pass_1_h_result = [] 

vo_pass_1_h_details = [
    vo_pass_1_h_roll, vo_pass_1_h_vote_type, vo_pass_1_h_acted_at, vo_pass_1_h_how,
    vo_pass_1_h_suspension, vo_pass_1_h_result
] 

In [87]:
compile_v_override_pass_house = action_compile_and_report(
    status='VETOED:OVERRIDE_PASS_OVER:HOUSE',
    output_list= veto_override_1_pass_house) 

Positive Status Detected
1.0


In [88]:
v_override_pass_h_details = detail_compile_and_report(
    status='VETOED:OVERRIDE_PASS_OVER:HOUSE',
    output_list= vo_pass_1_h_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 5.08907844618691e-05
Vote type: 5.08907844618691e-05
Acted at: 5.08907844618691e-05
How: 5.08907844618691e-05
Suspension: 0.0
Result: 5.08907844618691e-05


### Status: VETOED:OVERRIDE_PASS_OVER:SENATE

In [89]:
veto_override_1_pass_senate = []

vo_pass_1_s_roll = []
vo_pass_1_s_vote_type = []
vo_pass_1_s_acted_at = []
vo_pass_1_s_how = []
vo_pass_1_s_suspension = []
vo_pass_1_s_result = [] 

vo_pass_1_s_details = [
    vo_pass_1_s_roll, vo_pass_1_s_vote_type, vo_pass_1_s_acted_at, vo_pass_1_s_how,
    vo_pass_1_s_suspension, vo_pass_1_s_result
] 

In [90]:
compile_v_override_pass_senate = action_compile_and_report(
    status='VETOED:OVERRIDE_PASS_OVER:SENATE',
    output_list= veto_override_1_pass_senate) 

Positive Status Detected
1.0


In [91]:
v_override_pass_s_details = detail_compile_and_report(
    status='VETOED:OVERRIDE_PASS_OVER:SENATE',
    output_list= vo_pass_1_s_details)

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 2.3488054367016507e-05
Vote type: 2.3488054367016507e-05
Acted at: 2.3488054367016507e-05
How: 2.3488054367016507e-05
Suspension: 0.0
Result: 2.3488054367016507e-05


### Status: VETOED:OVERRIDE_FAIL_SECOND:HOUSE

In [96]:
veto_override_2_fail_house = []

vo_fail_2_h_roll = []
vo_fail_2_h_vote_type = []
vo_fail_2_h_acted_at = []
vo_fail_2_h_how = []
vo_fail_2_h_suspension = []
vo_fail_2_h_result = [] 

vo_fail_2_h_details = [
    vo_fail_2_h_roll, vo_fail_2_h_vote_type, vo_fail_2_h_acted_at, vo_fail_2_h_how,
    vo_fail_2_h_suspension, vo_fail_2_h_result
] 

In [97]:
compile_v_override_2_fail_house = action_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_SECOND:HOUSE',
    output_list= veto_override_2_fail_house) 

Positive Status Detected
1.0


In [98]:
v_override_2_fail_h_details = detail_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_SECOND:HOUSE',
    output_list= vo_fail_2_h_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 7.82935145567217e-06
Vote type: 7.82935145567217e-06
Acted at: 7.82935145567217e-06
How: 7.82935145567217e-06
Suspension: 0.0
Result: 7.82935145567217e-06


### Status: VETOED:OVERRIDE_FAIL_SECOND:SENATE

In [99]:
veto_override_2_fail_senate = []

vo_fail_2_s_roll = []
vo_fail_2_s_vote_type = []
vo_fail_2_s_acted_at = []
vo_fail_2_s_how = []
vo_fail_2_s_suspension = []
vo_fail_2_s_result = [] 

vo_fail_2_s_details = [
    vo_fail_2_s_roll, vo_fail_2_s_vote_type, vo_fail_2_s_acted_at, vo_fail_2_s_how,
    vo_fail_2_s_suspension, vo_fail_2_s_result
] 

In [100]:
compile_v_override_2_fail_senate = action_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_SECOND:SENATE',
    output_list= veto_override_2_fail_senate) 

Positive Status Detected
1.0


In [101]:
v_override_2_fail_s_details = detail_compile_and_report(
    status='VETOED:OVERRIDE_FAIL_SECOND:SENATE',
    output_list= vo_fail_2_s_details) 

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 3.523208155052476e-05
Vote type: 3.523208155052476e-05
Acted at: 3.523208155052476e-05
How: 3.523208155052476e-05
Suspension: 0.0
Result: 3.523208155052476e-05


### Status: ENACTED:VETO_OVERRIDE 

In [103]:
vo_enacted_roll = []
vo_enacted_vote_type = []
vo_enacted_acted_at = []
vo_enacted_how = []
vo_enacted_suspension = []
vo_enacted_result = [] 

vo_enacted_details = [
    vo_enacted_roll, vo_enacted_vote_type, vo_enacted_acted_at, vo_enacted_how,
    vo_enacted_suspension, vo_enacted_result
] 

In [102]:
enacted_veto_override = []
compile_enacted_v_override = action_compile_and_report(status='ENACTED:VETO_OVERRIDE',
                                                      output_list=enacted_veto_override)

Positive Status Detected
1.0


In [104]:
enacted_v_override_rolls = detail_compile_and_report(status='ENACTED:VETO_OVERRIDE',
                                                  output_list=vo_enacted_details)

Roll Number Positive Status Detected
Vote Type Positive Status Detected
Acted at Positive Status Detected
How Positive Status Detected
No Suspension Positive Status Detected
Result Positive Status Detected


Rolls: 3.914675727836085e-06
Vote type: 3.914675727836085e-06
Acted at: 6.654948737321344e-05
How: 3.914675727836085e-06
Suspension: 0.0
Result: 3.914675727836085e-06


### Status: 'ENACTED:TENDAYRULE' 

In [105]:
enacted_ten_day_rule = []
compile_enacted_10d_rule = action_compile_and_report(status='ENACTED:TENDAYRULE',
                                                      output_list=enacted_ten_day_rule)

Positive Status Detected
1.0


### Status: ENACTED:SIGNED 

In [106]:
enacted_signed = []
compile_enacted_signed = action_compile_and_report(status='ENACTED:SIGNED',
                                                      output_list=enacted_signed)

Positive Status Detected
1.0


In [107]:
congress_level_df = pd.read_csv('data_store/dataframes/full_congress_level_df2.csv') 

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (23,24,25,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [108]:
current_cols = list(congress_level_df.columns) 

In [109]:
current_cols 

['bill_id',
 'bill_type',
 'congress',
 'number',
 'action_count',
 'r_sponsor',
 'amendment_count',
 'committee_count',
 'cosponsor_count',
 'r_cosponsors',
 'd_cosponsors',
 'popular_title',
 'related_bill_count',
 'short_title',
 'subject_count',
 'title_count',
 'house_republicans',
 'house_democrats',
 'senate_republicans',
 'senate_democrats',
 'referral_committee_1',
 'referral_committee_2',
 'referral_committee_3',
 'markup_committee_1',
 'markup_committee_2',
 'markup_committee_3',
 'reporting_committee_1',
 'reporting_committee_2',
 'reporting_committee_3',
 'hearings_committee_1',
 'hearings_committee_2',
 'hearings_committee_3',
 'status',
 'bill_number',
 'summary',
 'major_label 1',
 'major_prob 1',
 'major_label 2',
 'major_prob 2',
 'major_label 3',
 'major_prob 3',
 'major_1_minor_label 1',
 'major_1_minor_prob 1',
 'major_1_minor_label 2',
 'major_1_minor_prob 2',
 'major_1_minor_label 3',
 'major_1_minor_prob 3',
 'major_2_minor_label 1',
 'major_2_minor_prob 1',
 'm

In [110]:
drop_list = ['referred',
 'reported',
 'failed_in_suspension',
 'cloture_failed',
 'failed_origin_house',
 'failed_origin_senate',
 'passover_house',
 'passover_senate',
 'failed_second_house',
 'failed_second_senate',
 'passed',
 'passback_house',
 'passback_senate',
 'passed_conference_house',
 'passed_conference_senate',
 'pocket_veto',
 'prov_kill_veto',
 'veto_override_fail_house',
 'veto_override_fail_senate',
 'veto_override_1_pass_house',
 'veto_override_1_pass_senate',
 'veto_override_2_fail_house',
 'veto_override_2_fail_senate',
 'enacted_veto_override',
 'enacted_ten_day_rule',
 'enacted_signed',
 'failed_in_suspension_rolls',
 'cloture_failed_rolls',
 'failed_origin_house_rolls',
 'failed_origin_senate_rolls',
 'passover_house_rolls',
 'passover_senate_rolls',
 'fail_second_house_rolls',
 'fail_second_senate_rolls',
 'passed_bill_rolls',
 'passback_house_rolls',
 'passback_senate_rolls',
 'conf_pass_h_rolls',
 'conf_pass_s_rolls',
 'v_override_fail_h_rolls',
 'v_override_fail_s_rolls',
 'v_override_pass_h_rolls',
 'v_override_pass_s_rolls',
 'v_override_2_fail_h_rolls',
 'v_override_2_fail_s_rolls',
 'enacted_v_override_rolls']

In [111]:
congress = congress_level_df.drop(drop_list, axis=1)

In [112]:
congress.columns 

Index(['bill_id', 'bill_type', 'congress', 'number', 'action_count',
       'r_sponsor', 'amendment_count', 'committee_count', 'cosponsor_count',
       'r_cosponsors', 'd_cosponsors', 'popular_title', 'related_bill_count',
       'short_title', 'subject_count', 'title_count', 'house_republicans',
       'house_democrats', 'senate_republicans', 'senate_democrats',
       'referral_committee_1', 'referral_committee_2', 'referral_committee_3',
       'markup_committee_1', 'markup_committee_2', 'markup_committee_3',
       'reporting_committee_1', 'reporting_committee_2',
       'reporting_committee_3', 'hearings_committee_1', 'hearings_committee_2',
       'hearings_committee_3', 'status', 'bill_number', 'summary',
       'major_label 1', 'major_prob 1', 'major_label 2', 'major_prob 2',
       'major_label 3', 'major_prob 3', 'major_1_minor_label 1',
       'major_1_minor_prob 1', 'major_1_minor_label 2', 'major_1_minor_prob 2',
       'major_1_minor_label 3', 'major_1_minor_prob 3',
   

In [45]:
outdated_df = pd.read_csv('data_store/dataframes/full_congress_level_df7.csv')

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (22,23,24,25,28,29,30,31,67,68,69,70,71,74,75,76,78,81,82,83,85,88,89,90,92,95,96,97,98,99,102,103,104,106,109,110,111,113,116,117,118,120,123,124,125,126,127,130,131,132,134,137,138,139,141,144,145,146,147,148,151,152,153,154,155,158,159,160,162,165,166,167,169,172,173,174,176,179,180,181,183,186,187,188,190,193,194,195,197,200,201,202,204,207,208,209,211) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
pre_vote_data_dict = {'pre_vote1_actions': df_pre_vote_1_actions,
                      'pre_vote2_actions': df_pre_vote_2_actions} 

In [50]:
pre_vote_action_df = pd.DataFrame(data=pre_vote_data_dict,
                                 columns=['pre_vote1_actions', 'pre_vote2_actions']) 

In [55]:
pre_vote_action_df['pre_vote1_actions'].unique() 

array([  0,   6,   5,   4,   9,   2,  10,   7,  -1,  13,   3,   8,  20,
        25,  17,  14,  99,  16,  11,  12,  30,  28,   1,  53,  37,  32,
        31,  84,  36,  18,  59,  19,  22,  26, 143, 121,  46,  34,  52,
        78,  23,  21,  15,  55,  27,  60,  24,  89,  40,  42,  38,  29,
        33,  58,  39,  51,  35,  48,  86,  44,  54,  41,  77,  66,  74,
        49,  95,  82,  43,  69,  65,  47,  92,  45,  63, 148,  75, 105,
        56,  93,  83,  87,  97,  64,  71,  76,  57,  72,  67,  62,  81,
       113,  80,  70,  91, 119,  50,  73,  88,  90, 110,  68, 120, 111,
        79,  61, 108,  98, 133, 101,  96, 115, 127, 102, 162, 103, 126,
        85, 130, 125, 156, 116, 337, 187, 144, 161, 118, 106, 147, 137,
       134,  94, 140, 132, 114, 107, 198, 104, 351, 323, 182, 146])

In [56]:
full_congress_df = pd.concat([outdated_df, pre_vote_action_df])

/home/william/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [58]:
full_congress_df['pre_vote1_actions'].nunique() 

142

In [59]:
full_congress_df.to_csv('data_store/dataframes/full_congress_level_df8.csv', index=False)